In [1]:
import pandas as pd



# 1. 분리 문자(Separator)를 세미콜론(;)으로 지정
df = pd.read_csv('csv 데이터\clean\contents.csv', encoding='cp949')



In [2]:
df.iloc[200:210]

,content_id,domain,master_title,original_title,poster_image_url,release_year,created_at,updated_at,synopsis
200,199,AV,도쿄 구울,東京?種ト?キョ?グ?ル,https://image.tmdb.org/t/p/w500/nkDtqLGG7J7L7Y...,NaN,2025-10-29,2025-10-29,"""군집 속에 섞여 인간을 사냥하고 그 죽은 고기를 먹는 괴인.\n\n우리들은 그들을..."
201,200,AV,마이 리틀 포니: 우정은 마법,My Little Pony: Friendship Is Magic,https://image.tmdb.org/t/p/w500/iqpxbJX66AjG7i...,NaN,2025-10-29,2025-10-29,포니 트와일라잇 스파클과 다섯 명의 포니가 가까운 친구가 되면서 우정에 생긴 문제들...
202,201,GAME,Counter-Strike,NaN,https://shared.akamai.steamstatic.com/store_it...,NaN,2025-10-29,2025-10-29,세계 최고의 온라인 액션 게임을 즐기세요. 전 세계적으로 유명한 팀 기반의 게임에서...
203,202,GAME,Team Fortress Classic,NaN,https://shared.akamai.steamstatic.com/store_it...,NaN,2025-10-29,2025-10-29,전 세계에서 가장 유명한 온라인 액션 게임 중 하나인 Team Fortress Cl...
204,203,GAME,Day of Defeat,NaN,https://shared.akamai.steamstatic.com/store_it...,NaN,2025-10-29,2025-10-29,제 2차 세계 대전의 유럽 작전 지역에 배치된 생생한 추축국 대 연합군 팀플레이에 ...
205,204,GAME,Deathmatch Classic,NaN,https://shared.akamai.steamstatic.com/store_it...,NaN,2025-10-29,2025-10-29,Deathmatch Classic(DMC)에서 빠른 템포의 멀티플레이어 게임을 즐기...
206,205,GAME,Half-Life: Opposing Force,NaN,https://shared.akamai.steamstatic.com/store_it...,NaN,2025-10-29,2025-10-29,완전히 새로운 경험의 싱글플레이어 액션을 체험해보세요. 사납고 맹렬한 외계인 적군을...
207,206,GAME,Ricochet,NaN,https://shared.akamai.steamstatic.com/store_it...,NaN,2025-10-29,2025-10-29,50개 이상의 출판물에서 올해의 게임으로 선정된 Valve의 데뷔 타이틀은 액션과 ...
208,207,GAME,Half-Life,NaN,https://shared.akamai.steamstatic.com/store_it...,NaN,2025-10-29,2025-10-29,"50개 이상의 언론사에서 올해의 게임으로 선정된 Valve의 데뷔 타이틀로서, 수상..."
209,208,GAME,Counter-Strike: Condition Zero,NaN,https://shared.akamai.steamstatic.com/store_it...,NaN,2025-10-29,2025-10-29,Nuclearvision Entertainment가 제작한 Codename Gord...


In [2]:
# -*- coding: cp949 -*-
import csv
import json

# ===== 파일 경로 직접 수정해서 사용 =====
INPUT_TXT = r"csv 데이터\av_contents_txt.txt"   # 원본 txt
OUTPUT_CSV = r"csv 데이터\clean\av_contents.csv"  # 결과 csv

# 한 줄에 최대 몇 개의 장르를 펼칠지 (0~3 → 총 4개)
MAX_GENRES = 4

# ===== 장르 → TMDB ID 매핑 =====
# 키는 JSON 안에 들어있는 "한글 장르명" 그대로
GENRE_MAP = {
    u"공포":       (27,    u"공포"),
    u"스릴러":     (53,    u"스릴러"),
    u"전쟁":       (10752, u"전쟁"),
    u"액션":       (28,    u"액션"),
    u"범죄":       (80,    u"범죄"),
    u"판타지":     (14,    u"판타지"),
    u"음악":       (10402, u"음악"),
    u"코미디":     (35,    u"코미디"),
    u"애니메이션": (16,    u"애니메이션"),
    u"드라마":     (18,    u"드라마"),
    u"가족":       (10751, u"가족"),
    u"미스터리":   (9648,  u"미스터리"),
    u"역사":       (36,    u"역사"),
    u"로맨스":     (10749, u"로맨스"),
    u"SF":        (878,   u"SF"),
    u"다큐멘터리": (99,    u"다큐멘터리"),
    u"모험":       (12,    u"모험"),
}


def parse_line_to_json(line):
    """
    한 줄:
    "{""release_date"":""2025-10-22"",""tmdb_id"":1306525,...}"
    이런 형식을 실제 JSON 문자열로 복원해서 dict 로 변환
    """
    line = line.strip()
    if not line:
        return None

    # 첫 줄 "row_to_json" 헤더면 건너뛰기
    if line.lower().startswith('"row_to_json"'):
        return None

    # 맨 앞/뒤 큰따옴표 제거
    if line.startswith('"') and line.endswith('"'):
        line = line[1:-1]

    # "" -> " 로 변경 (CSV 스타일 escape)
    line = line.replace('""', '"')

    try:
        obj = json.loads(line)
    except Exception as e:
        print("JSON 파싱 실패:", e)
        print("문제된 라인:", line)
        return None

    return obj


def main():
    # ANSI(cp949) 로 읽고 쓰기
    with open(INPUT_TXT, "r", encoding="cp949") as f_in, \
         open(OUTPUT_CSV, "w", encoding="cp949", newline="") as f_out:

        writer = csv.writer(f_out)

        # ===== 헤더 작성 =====
        header = [
            "av_type",
            "content_id",
        ]
        for i in range(MAX_GENRES):
            header.append("genres.tmdb_genres.%d.id" % i)
            header.append("genres.tmdb_genres.%d.name" % i)
        header.extend(["release_date", "tmdb_id"])
        writer.writerow(header)

        # ===== 본문 =====
        for raw_line in f_in:
            obj = parse_line_to_json(raw_line)
            if obj is None:
                continue

            av_type = (obj.get("av_type") or "").upper()  # movie -> MOVIE
            content_id = obj.get("content_id", "")
            release_date = obj.get("release_date", "")
            tmdb_id = obj.get("tmdb_id", "")
            genres = obj.get("genres") or []

            row = [av_type, content_id]

            # 장르 최대 4개까지 펼치기
            for i in range(MAX_GENRES):
                if i < len(genres):
                    g = genres[i]
                    # 매핑 있으면 id/name, 없으면 id 공백 + 원본 이름
                    if g in GENRE_MAP:
                        gid, gname = GENRE_MAP[g]
                    else:
                        gid, gname = "", g
                    row.extend([gid, gname])
                else:
                    # 부족한 칸은 빈칸
                    row.extend(["", ""])

            row.extend([release_date, tmdb_id])
            writer.writerow(row)


if __name__ == "__main__":
    main()


In [ ]:
# -*- coding: cp949 -*-
import csv
import json
import os


INPUT_TXT = r"csv 데이터\contents_txt.txt"   # 원본 txt
OUTPUT_CSV = r"csv 데이터\clean\contents.csv"  # 결과 csv

# ===== JSON 한 줄 파싱 유틸 =====
def parse_line_to_json(line):
    line = line.strip()
    if not line:
        return None

    # 첫 줄 "row_to_json" 헤더면 스킵
    if line.lower().startswith('"row_to_json"'):
        return None

    # 맨 앞/뒤 큰따옴표 제거
    if line.startswith('"') and line.endswith('"'):
        line = line[1:-1]

    # "" -> " (CSV 스타일 escape 제거)
    line = line.replace('""', '"')

    try:
        return json.loads(line)
    except Exception as e:
        print("JSON 파싱 실패:", e)
        print("문제된 라인:", line)
        return None


def main():
    # 입력 파일 체크
    if not os.path.exists(INPUT_TXT):
        print("❌ INPUT_TXT 파일 없음")
        print("INPUT_TXT =", INPUT_TXT)
        return

    # 출력 폴더 없으면 생성
    out_dir = os.path.dirname(OUTPUT_CSV)
    if out_dir and not os.path.exists(out_dir):
        os.makedirs(out_dir, exist_ok=True)

    with open(INPUT_TXT, "r", encoding="cp949") as f_in, \
         open(OUTPUT_CSV, "w", encoding="cp949", newline="") as f_out:

        writer = csv.writer(f_out)

        # ===== 헤더 =====
        header = [
            "content_id",
            "domain",
            "master_title",
            "original_title",
            "poster_image_url",
            "release_year",
            "created_at",
            "updated_at",
            "synopsis",
        ]
        writer.writerow(header)

        # ===== 본문 =====
        for raw_line in f_in:
            obj = parse_line_to_json(raw_line)
            if obj is None:
                continue

            content_id       = obj.get("content_id", "")
            domain           = obj.get("domain", "")
            master_title     = obj.get("master_title", "")
            original_title   = obj.get("original_title", "")
            poster_image_url = obj.get("poster_image_url", "")
            release_date     = obj.get("release_date")  # null이면 None
            created_at       = obj.get("created_at", "")
            updated_at       = obj.get("updated_at", "")
            synopsis         = obj.get("synopsis", "")

            # release_year: release_date가 "2025-10-29" 이런 형식이면 앞 4자리
            if release_date:
                release_year = str(release_date)[:4]
            else:
                release_year = ""

            # created_at / updated_at 은 날짜만 쓰고 싶으면 이처럼 자르기
            if created_at:
                created_at_out = str(created_at).split("T", 1)[0]
            else:
                created_at_out = ""

            if updated_at:
                updated_at_out = str(updated_at).split("T", 1)[0]
            else:
                updated_at_out = ""

            row = [
                content_id,
                domain,
                master_title,
                original_title,
                poster_image_url,
                release_year,
                created_at_out,
                updated_at_out,
                synopsis,
            ]
            writer.writerow(row)

    print("✅ 완료! ->", OUTPUT_CSV)


if __name__ == "__main__":
    main()


✅ 완료! -> csv 데이터\clean\contents.csv


In [4]:
# -*- coding: cp949 -*-
import csv
import json
import os

# ===== 경로만 네 환경에 맞게 수정 =====
INPUT_TXT = r"csv 데이터\game_contents_txt.txt"   # 원본 txt
OUTPUT_CSV = r"csv 데이터\clean\game_contents.csv"  # 결과 csv

def parse_line_to_json(line):
    line = line.strip()
    if not line:
        return None

    # 첫 줄 "row_to_json" 헤더 스킵
    if line.lower().startswith('"row_to_json"'):
        return None

    # 맨 앞/뒤 큰따옴표 제거
    if line.startswith('"') and line.endswith('"'):
        line = line[1:-1]

    # "" -> " (CSV escape 제거)
    line = line.replace('""', '"')

    try:
        return json.loads(line)
    except Exception as e:
        print("JSON 파싱 실패:", e)
        print("문제된 라인:", line)
        return None


def main():
    # 입력 파일 체크
    if not os.path.exists(INPUT_TXT):
        print("❌ INPUT_TXT 파일 없음")
        print("INPUT_TXT =", INPUT_TXT)
        print("현재 작업 디렉터리 =", os.getcwd())
        return

    # 출력 폴더 없으면 만들기
    out_dir = os.path.dirname(OUTPUT_CSV)
    if out_dir and not os.path.exists(out_dir):
        os.makedirs(out_dir, exist_ok=True)

    with open(INPUT_TXT, "r", encoding="cp949") as f_in, \
         open(OUTPUT_CSV, "w", encoding="cp949", newline="") as f_out:

        writer = csv.writer(f_out)

        # ===== 헤더 =====
        header = [
            "content_id",
            "developer",
            "publisher",
            "release_date",
            "platform_mac",
            "platform_linux",
            "platform_windows",
            "genres_str",
        ]
        writer.writerow(header)

        # ===== 본문 =====
        for raw_line in f_in:
            obj = parse_line_to_json(raw_line)
            if obj is None:
                continue

            content_id   = obj.get("content_id", "")
            developer    = obj.get("developer", "")
            publisher    = obj.get("publisher", "")
            release_date = obj.get("release_date", "")  # "2000-11-01" 같은 형식

            platforms = obj.get("platforms") or {}
            platform_mac    = bool(platforms.get("mac", False))
            platform_linux  = bool(platforms.get("linux", False))
            platform_window = bool(platforms.get("windows", False))

            genres = obj.get("genres") or []
            # 여러 개면 "액션|RPG" 이런 식으로 붙이기
            genres_str = "|".join(genres)

            row = [
                content_id,
                developer,
                publisher,
                release_date,
                platform_mac,
                platform_linux,
                platform_window,
                genres_str,
            ]
            writer.writerow(row)

    print("✅ 완료! ->", OUTPUT_CSV)


if __name__ == "__main__":
    main()


✅ 완료! -> csv 데이터\clean\game_contents.csv


In [7]:
# -*- coding: cp949 -*-
import csv
import json
import os
import re  # ✅ 추가: 잘못된 \ escape 처리용

# ===== 경로만 네 환경에 맞게 수정해서 사용 =====
INPUT_TXT = r"csv 데이터\platform_data_txt.txt"   # 원본 txt
OUTPUT_CSV = r"csv 데이터\clean\platform_data.csv"  # 결과 csv

def parse_line_to_json(line):
    line = line.strip()
    if not line:
        return None

    # 첫 줄 "row_to_json" 헤더 스킵
    if line.lower().startswith('"row_to_json"'):
        return None

    # 맨 앞/뒤 큰따옴표 제거 (있을 때만)
    if line.startswith('"') and line.endswith('"'):
        line = line[1:-1]
        # "" -> " (CSV escape 제거)
        line = line.replace('""', '"')

    # ✅ 문제: "\ 5,500" 같은 잘못된 JSON 이스케이프 수정
    #   \ 뒤에 공백이 오는 경우 → \\ + 공백 으로 바꿔서 유효하게 만든다.
    #   예: "\ 5,500" -> "\\ 5,500"
    line = re.sub(r'\\( )', r'\\\\\1', line)

    try:
        return json.loads(line)
    except Exception as e:
        print("JSON 파싱 실패:", e)
        print("문제된 라인:", line)
        return None


def main():
    # 입력 파일 체크
    if not os.path.exists(INPUT_TXT):
        print("❌ INPUT_TXT 파일 없음")
        print("INPUT_TXT =", INPUT_TXT)
        print("현재 작업 디렉터리 =", os.getcwd())
        return

    # 출력 폴더 없으면 생성
    out_dir = os.path.dirname(OUTPUT_CSV)
    if out_dir and not os.path.exists(out_dir):
        os.makedirs(out_dir, exist_ok=True)

    with open(INPUT_TXT, "r", encoding="cp949") as f_in, \
         open(OUTPUT_CSV, "w", encoding="cp949", newline="") as f_out:

        writer = csv.writer(f_out)

        # ===== 헤더 =====
        header = [
            "platform_data_id",
            "content_id",
            "platform_name",
            "platform_specific_id",
            "url",
            "last_seen_at",
            "runtime",
            "season_count",
            "cast_count",
            "crew_count",
            "watch_providers_keys",
        ]
        writer.writerow(header)

        # ===== 본문 =====
        for raw_line in f_in:
            obj = parse_line_to_json(raw_line)
            if obj is None:
                continue

            platform_data_id      = obj.get("platform_data_id", "")
            content_id            = obj.get("content_id", "")
            platform_name_raw     = obj.get("platform_name", "")
            platform_specific_id  = obj.get("platform_specific_id", "")
            url                   = obj.get("url", "")
            last_seen_at          = obj.get("last_seen_at", "")
            attrs                 = obj.get("attributes") or {}

            # platform_name: TMDB_MOVIE → TMDB 이런 식으로 줄이기
            if isinstance(platform_name_raw, str) and platform_name_raw.startswith("TMDB"):
                platform_name = "TMDB"
            else:
                platform_name = platform_name_raw

            # last_seen_at: 날짜만 남기기 (예: 2025-10-29T... → 2025-10-29)
            if last_seen_at:
                last_seen_out = str(last_seen_at).split("T", 1)[0]
            else:
                last_seen_out = ""

            # runtime (Steam 데이터는 없을 수도 있음)
            runtime = attrs.get("runtime", "")

            # season_count: TV 쪽에만 쓰일 수 있으니 기본 0.0
            season_count = attrs.get("season_count", 0.0)

            # cast / crew 리스트 (TMDB 쪽에만 있을 수 있음)
            cast_list = attrs.get("cast") or []
            crew_list = attrs.get("crew") or []
            cast_count = len(cast_list)
            crew_count = len(crew_list)

            # watch_providers: 리스트를 "|" 로 합치기 (Steam은 없고 TMDB만 있을 가능성 큼)
            wp_list = attrs.get("watch_providers") or []
            watch_providers_keys = "|".join(wp_list) if wp_list else ""

            row = [
                platform_data_id,
                content_id,
                platform_name,
                platform_specific_id,
                url,
                last_seen_out,
                runtime,
                season_count,
                cast_count,
                crew_count,
                watch_providers_keys,
            ]
            writer.writerow(row)

    print("✅ 완료! ->", OUTPUT_CSV)


if __name__ == "__main__":
    main()


✅ 완료! -> csv 데이터\clean\platform_data.csv


In [11]:
# -*- coding: utf-8 -*-
import csv
import json
import os
import re  # 잘못된 \ escape 처리용

# ===== 경로만 네 환경에 맞게 수정해서 사용 =====
INPUT_TXT  = r"csv 데이터\raw_item_txt.txt"           # 지금 같은 row_to_json 텍스트
OUTPUT_CSV = r"csv 데이터\clean\raw_item.csv"    # 만들 CSV


def parse_line_to_json(line):
    line = line.strip()
    if not line:
        return None

    # 첫 줄 "row_to_json" 헤더 스킵
    if line.lower().startswith('"row_to_json"'):
        return None

    # CSV에서 따옴표로 한 번 더 감싸진 경우 처리
    if line.startswith('"') and line.endswith('"'):
        line = line[1:-1]
        # "" -> " (CSV escape 제거)
        line = line.replace('""', '"')

    # Steam 쪽에서 종종 생기는 잘못된 "\ 5,500" 같은 패턴 수정
    # \ 뒤에 공백이 오는 경우 → \\ + 공백 으로 바꿔서 유효한 JSON으로
    line = re.sub(r'\\( )', r'\\\\\1', line)

    try:
        return json.loads(line)
    except Exception as e:
        print("JSON 파싱 실패:", e)
        print("문제된 라인:", line[:300] + "..." if len(line) > 300 else line)
        return None


def main():
    # 입력 파일 체크
    if not os.path.exists(INPUT_TXT):
        print("❌ INPUT_TXT 파일 없음")
        print("INPUT_TXT =", INPUT_TXT)
        print("현재 작업 디렉터리 =", os.getcwd())
        return

    # 출력 폴더 없으면 생성
    out_dir = os.path.dirname(OUTPUT_CSV)
    if out_dir and not os.path.exists(out_dir):
        os.makedirs(out_dir, exist_ok=True)

    # ✅ 입출력 모두 UTF-8 로
    with open(INPUT_TXT, "r", encoding="utf-8") as f_in, \
         open(OUTPUT_CSV, "w", encoding="utf-8", newline="") as f_out:

        writer = csv.writer(f_out)

        # ===== 헤더 =====
        header = [
            "raw_id",
            "domain",
            "platform_name",
            "platform_specific_id",
            "name",
            "content_type",
            "genres_str",
            "is_free",
            "platform_mac",
            "platform_linux",
            "platform_windows",
            "release_date",
            "fetched_at",
            "processed_at",
            "url",
        ]
        writer.writerow(header)

        # ===== 본문 =====
        for raw_line in f_in:
            obj = parse_line_to_json(raw_line)
            if obj is None:
                continue

            raw_id               = obj.get("raw_id", "")
            domain               = obj.get("domain", "")
            platform_name        = obj.get("platform_name", "")
            platform_specific_id = obj.get("platform_specific_id", "")
            url                  = obj.get("url", "")
            fetched_at           = obj.get("fetched_at", "")
            processed_at         = obj.get("processed_at", "")
            payload              = obj.get("source_payload") or {}

            # 날짜만 (YYYY-MM-DD) 로 자르기
            if fetched_at:
                fetched_at_out = str(fetched_at).split("T", 1)[0]
            else:
                fetched_at_out = ""

            if processed_at:
                processed_at_out = str(processed_at).split("T", 1)[0]
            else:
                processed_at_out = ""

            # 이름 / 타입
            name = (
                payload.get("name", "")
                or payload.get("original_name", "")
                or ""
            )
            content_type = payload.get("type", "") or payload.get("av_type", "")

            # 장르
            genres = payload.get("genres") or []
            genres_str = "|".join(genres)

            # 무료 여부
            is_free = payload.get("is_free", "")

            # 플랫폼 (Steam GAME 쪽)
            platforms = payload.get("platforms") or {}
            platform_mac     = bool(platforms.get("mac", False))
            platform_linux   = bool(platforms.get("linux", False))
            platform_windows = bool(platforms.get("windows", False))

            # 출시일:
            #  - Steam: source_payload.release_date.date (dict)
            #  - TMDB_TV: source_payload.first_air_date (string)
            release_date = ""
            if "release_date" in payload:
                rd = payload.get("release_date")
                if isinstance(rd, dict):
                    release_date = rd.get("date", "") or ""
                else:
                    release_date = rd or ""
            elif "first_air_date" in payload:
                release_date = payload.get("first_air_date", "")

            row = [
                raw_id,
                domain,
                platform_name,
                platform_specific_id,
                name,
                content_type,
                genres_str,
                is_free,
                platform_mac,
                platform_linux,
                platform_windows,
                release_date,
                fetched_at_out,
                processed_at_out,
                url,
            ]
            writer.writerow(row)

    print("✅ 완료! ->", OUTPUT_CSV)


if __name__ == "__main__":
    main()


✅ 완료! -> csv 데이터\clean\raw_item.csv


In [12]:
# -*- coding: cp949 -*-
import csv
import json
import os

# ===== 파일 경로 수정 =====
INPUT_TXT  = r"csv 데이터\webnovel_contents_txt.txt"           # 위에 있는 row_to_json 텍스트 파일
OUTPUT_CSV = r"csv 데이터\clean\webnovel_contents.csv"    # 결과 CSV

def parse_line_to_json(line):
    line = line.strip()
    if not line:
        return None

    # 첫 줄 "row_to_json" 헤더 스킵
    if line.lower().startswith('"row_to_json"'):
        return None

    # 바깥에 한번 더 "로 감싸져 있을 수 있어서 제거
    if line.startswith('"') and line.endswith('"'):
        line = line[1:-1]
        # "" -> " (CSV escape 제거)
        line = line.replace('""', '"')

    try:
        return json.loads(line)
    except Exception as e:
        print("JSON 파싱 실패:", e)
        print("문제된 라인:", line)
        return None


def main():
    # 입력 파일 존재 확인
    if not os.path.exists(INPUT_TXT):
        print("❌ INPUT_TXT 파일 없음")
        print("INPUT_TXT =", INPUT_TXT)
        print("현재 작업 디렉터리 =", os.getcwd())
        return

    # 출력 폴더 없으면 생성
    out_dir = os.path.dirname(OUTPUT_CSV)
    if out_dir and not os.path.exists(out_dir):
        os.makedirs(out_dir, exist_ok=True)

    with open(INPUT_TXT, "r", encoding="cp949") as f_in, \
         open(OUTPUT_CSV, "w", encoding="cp949", newline="") as f_out:

        writer = csv.writer(f_out)

        # ===== 헤더 =====
        header = [
            "content_id",
            "age_rating",
            "author",
            "publisher",
            "genres_str",
            "started_at",
        ]
        writer.writerow(header)

        # ===== 본문 =====
        for raw_line in f_in:
            obj = parse_line_to_json(raw_line)
            if obj is None:
                continue

            content_id  = obj.get("content_id", "")
            age_rating  = obj.get("age_rating", "")
            author      = obj.get("author", "")
            publisher   = obj.get("publisher", "")
            genres      = obj.get("genres") or []
            started_at  = obj.get("started_at", "")

            genres_str = "|".join(genres)

            row = [
                content_id,
                age_rating,
                author,
                publisher,
                genres_str,
                started_at,
            ]
            writer.writerow(row)

    print("✅ 완료! ->", OUTPUT_CSV)


if __name__ == "__main__":
    main()


✅ 완료! -> csv 데이터\clean\webnovel_contents.csv


In [17]:
import os
import pandas as pd

# ──────────────────────────────
# 1. 기본 경로 설정
# ──────────────────────────────
base_dir = r"csv 데이터\clean"
output_file = os.path.join(base_dir, "csv_summary.txt")

# ──────────────────────────────
# 2. CSV 목록 불러오기
# ──────────────────────────────
csv_files = [f for f in os.listdir(base_dir) if f.lower().endswith(".csv")]

if not csv_files:
    print("❌ CSV 파일이 없습니다.")
    exit()

# ──────────────────────────────
# 3. 결과 파일 초기화
# ──────────────────────────────
with open(output_file, "w", encoding="utf-8") as f_out:
    f_out.write("📊 CSV 파일 요약 보고서\n")
    f_out.write("=" * 60 + "\n\n")

# ──────────────────────────────
# 4. 각 CSV 파일에 대한 정보 수집
# ──────────────────────────────
for file_name in csv_files:
    file_path = os.path.join(base_dir, file_name)
    print(f"처리 중: {file_name}")

    # 파일 읽기 (UTF-8 → CP949 순으로 시도)
    try:
        df = pd.read_csv(file_path, encoding="utf-8")
    except UnicodeDecodeError:
        df = pd.read_csv(file_path, encoding="cp949")
    except Exception as e:
        with open(output_file, "a", encoding="utf-8") as f_out:
            f_out.write(f"⚠️ {file_name} 읽기 오류: {e}\n\n")
        continue

    # ── 상위 5개 샘플만 표시 ──
    sample = df.head(5)

    # ── 결과 작성 ──
    with open(output_file, "a", encoding="utf-8") as f_out:
        f_out.write(f"📄 파일명: {file_name}\n")
        f_out.write(f"컬럼 ({len(df.columns)}개): {list(df.columns)}\n\n")
        f_out.write("샘플 5행:\n")
        f_out.write(sample.to_string(index=False))
        f_out.write("\n" + "-" * 60 + "\n\n")

# ──────────────────────────────
# 5. 완료 메시지
# ──────────────────────────────
print(f"✅ 요약 파일 생성 완료: {output_file}")


처리 중: av_contents.csv
처리 중: contents.csv
처리 중: content_raw_genres.csv
처리 중: game_contents.csv
처리 중: graph_edges_bipartite.csv
처리 중: graph_edges_item_item.csv
처리 중: graph_nodes.csv
처리 중: item_embeddings_torch.csv
처리 중: meta_nodes.csv
처리 중: platform_data.csv
처리 중: raw_item.csv
처리 중: webnovel_contents.csv
✅ 요약 파일 생성 완료: csv 데이터\clean\csv_summary.txt


# content_raw_genres 만들기

In [50]:
# -*- coding: utf-8 -*-

import os
import re
import pandas as pd
from typing import Optional, List

BASE = r"csv 데이터\clean"
OLD_GENRES = os.path.join(BASE, "content_raw_genres.csv")  # 기존 파일
RAW_ITEM   = os.path.join(BASE, "raw_item.csv")            # raw_id, genres_str 있는 파일

# ⚠️ 덮어쓰고 싶으면 OUT_GENRES를 OLD_GENRES로 바꿔도 됨
OUT_GENRES = os.path.join(BASE, "content_raw_genres.csv")

def read_csv_retry(path, encodings=("utf-8-sig","utf-8","cp949","euc-kr","latin1"), **kw) -> Optional[pd.DataFrame]:
    if not os.path.exists(path):
        return None
    last = None
    for enc in encodings:
        try:
            return pd.read_csv(path, encoding=enc, **kw)
        except Exception as e:
            last = e
    raise last

def parse_genres(genres_str: str) -> List[str]:
    """
    '드라마|범죄|미스터리' 처럼 생긴 문자열을 ['드라마','범죄','미스터리']로 파싱.
    없으면 [] 리턴.
    """
    if not isinstance(genres_str, str):
        return []
    genres_str = genres_str.strip()
    if not genres_str:
        return []
    parts = re.split(r"[|,/·]", genres_str)
    parts = [g.strip() for g in parts if g.strip()]
    return parts

def main():
    # 1) 기존 content_raw_genres.csv
    cg = read_csv_retry(OLD_GENRES)
    if cg is None or cg.empty:
        raise RuntimeError("content_raw_genres.csv 를 찾지 못했거나 비어 있습니다.")

    cg = cg.copy()
    cg.columns = [c.strip() for c in cg.columns]

    needed_cg = ["content_id", "source", "raw_genre"]
    miss_cg = [c for c in needed_cg if c not in cg.columns]
    if miss_cg:
        raise RuntimeError("content_raw_genres.csv 에 다음 컬럼이 필요합니다: " + ", ".join(miss_cg))

    cg["content_id"] = pd.to_numeric(cg["content_id"], errors="coerce").astype("Int64")
    cg = cg[cg["content_id"].notna()].copy()
    cg["content_id"] = cg["content_id"].astype(int)

    # 2) raw_item.csv
    ri = read_csv_retry(RAW_ITEM)
    if ri is None or ri.empty:
        raise RuntimeError("raw_item.csv 를 찾지 못했거나 비어 있습니다.")

    ri = ri.copy()
    ri.columns = [c.strip() for c in ri.columns]

    needed_ri = ["raw_id", "genres_str"]
    miss_ri = [c for c in needed_ri if c not in ri.columns]
    if miss_ri:
        raise RuntimeError("raw_item.csv 에 다음 컬럼이 필요합니다: " + ", ".join(miss_ri))

    ri["raw_id"] = pd.to_numeric(ri["raw_id"], errors="coerce").astype("Int64")
    ri = ri[ri["raw_id"].notna()].copy()
    ri["raw_id"] = ri["raw_id"].astype(int)

    # 3) content_id == raw_id 로 join 해서 genres_str 붙이기
    merged = cg.merge(
        ri[["raw_id", "genres_str"]],
        left_on="content_id",
        right_on="raw_id",
        how="left"
    )

    # 4) raw_genre_1,2,3 생성
    new_rows = []
    for _, row in merged.iterrows():
        cid = int(row["content_id"])
        source = str(row["source"]).strip()

        base_genre = str(row["raw_genre"]).strip()  # 기존 한 개짜리 raw_genre
        genres_from_str = parse_genres(row.get("genres_str", ""))

        # 우선순위:
        # 1) genres_str에서 파싱한 장르들을 우선 사용
        # 2) 기존 raw_genre가 비어있지 않고, 목록에 없으면 맨 앞에 추가
        genres = list(genres_from_str)
        if base_genre and base_genre not in genres:
            genres.insert(0, base_genre)

        # 최대 3개까지만 사용
        g1 = genres[0] if len(genres) > 0 else ""
        g2 = genres[1] if len(genres) > 1 else ""
        g3 = genres[2] if len(genres) > 2 else ""

        new_rows.append((cid, source, g1, g2, g3))

    out = pd.DataFrame(new_rows, columns=["content_id", "source", "raw_genre_1", "raw_genre_2", "raw_genre_3"])

    print(f"✅ 변환된 행 수: {len(out)}")
    out.to_csv(OUT_GENRES, index=False, encoding="utf-8-sig")
    print(f"✅ 저장 완료: {OUT_GENRES}")

if __name__ == "__main__":
    main()


✅ 변환된 행 수: 630
✅ 저장 완료: csv 데이터\clean\content_raw_genres.csv


In [9]:
import os
import io
import json
import re
import pandas as pd
import numpy as np
from datetime import datetime

# ====== 설정 ======
BASE_DIR = r"C:\Users\LG\Desktop\2025-2\A.O.D\db_export"
OUT_DIR  = os.path.join(BASE_DIR, "clean")
os.makedirs(OUT_DIR, exist_ok=True)

# 인코딩 시도 순서
ENCODINGS = ["utf-8-sig", "utf-8", "cp949", "euc-kr", "latin1"]

# 괄호 내부 콤마 치환용 (토크나이징 오류 회피)
COMMA_PLACEHOLDER = "⟐"

DATE_COL_PAT = re.compile(r"(date|_at)$", re.IGNORECASE)  # release_date, created_at 등


# ========== 유틸 ==========

def read_text_with_encodings(path: str) -> str:
    for enc in ENCODINGS:
        try:
            with io.open(path, "r", encoding=enc, errors="strict") as f:
                return f.read()
        except Exception:
            continue
    # 최후: 에러 무시 라틴1
    with io.open(path, "r", encoding="latin1", errors="ignore") as f:
        return f.read()

def protect_commas_inside_brackets(line: str) -> str:
    """{},[],() 내부의 콤마를 임시 치환해서 필드 수 불일치 방지"""
    res, stack = [], []
    for ch in line:
        if ch in "{[(":
            stack.append(ch); res.append(ch)
        elif ch in "}])":
            if stack: stack.pop()
            res.append(ch)
        else:
            if ch == "," and stack:
                res.append(COMMA_PLACEHOLDER)
            else:
                res.append(ch)
    return "".join(res)

def make_safe_temp_csv_text(raw_text: str) -> str:
    return "\n".join(protect_commas_inside_brackets(ln) for ln in raw_text.splitlines())

def try_read_csv(path: str) -> pd.DataFrame:
    """일반→치환텍스트→bad_lines skip 순서로 로드 시도"""
    raw = read_text_with_encodings(path)

    # 1) 일반 시도
    for enc in ENCODINGS + ["latin1"]:
        try:
            return pd.read_csv(io.StringIO(raw), encoding=enc, engine="python")
        except Exception:
            pass

    # 2) 괄호 내부 콤마 치환 후 재시도
    safe = make_safe_temp_csv_text(raw)
    for enc in ENCODINGS + ["latin1"]:
        try:
            df = pd.read_csv(io.StringIO(safe), encoding=enc, engine="python")
            # 치환 원복
            df = df.applymap(lambda x: x.replace(COMMA_PLACEHOLDER, ",") if isinstance(x, str) else x)
            return df
        except Exception:
            pass

    # 3) 최후: bad lines skip
    df = pd.read_csv(io.StringIO(safe), engine="python", on_bad_lines="skip")
    df = df.applymap(lambda x: x.replace(COMMA_PLACEHOLDER, ",") if isinstance(x, str) else x)
    return df

def parse_json_cell(val):
    """CSV 안 JSON 문자열(이중따옴표 이스케이프 포함)을 dict/list로 파싱(실패 시 원문 유지)"""
    if not isinstance(val, str): return val
    txt = val.strip()
    if not txt: return val
    # "{""k"":1}" -> {"k":1}
    if txt.startswith('"') and txt.endswith('"'):
        txt = txt[1:-1]
    txt = txt.replace('""', '"')
    if (txt.startswith("{") and txt.endswith("}")) or (txt.startswith("[") and txt.endswith("]")):
        try:
            return json.loads(txt)
        except Exception:
            return val
    return val

def is_jsonish_series(s: pd.Series) -> bool:
    """시리즈의 앞부분을 보고 JSON 문자열 비율이 높으면 True"""
    sample = s.dropna().astype(str).head(20).tolist()
    if not sample: return False
    cnt = 0
    for v in sample:
        v = v.strip().strip('"')
        if (v.startswith("{") and v.endswith("}")) or (v.startswith("[") and v.endswith("]")):
            cnt += 1
    return cnt >= max(3, len(sample)//4)

def normalize_df(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()

    # 1) JSON 문자열로 보이는 컬럼 파싱
    for col in out.columns:
        try:
            if out[col].dtype == "object" and is_jsonish_series(out[col]):
                out[col] = out[col].apply(parse_json_cell)
        except Exception:
            # 컬럼 단위 실패는 무시하고 진행
            pass

    # 2) 날짜 컬럼 ISO 문자열화
    for col in out.columns:
        if DATE_COL_PAT.search(col):
            try:
                out[col] = pd.to_datetime(out[col], errors="coerce").dt.strftime("%Y-%m-%d")
            except Exception:
                pass

    # 3) NaN/NaT -> None, numpy 타입 -> 파이썬 기본 타입화
    out = out.where(pd.notnull(out), None)

    return out

def df_to_json_records(df: pd.DataFrame):
    """JSON 직렬화 친화적인 records 생성"""
    # pandas 기본 dict 변환
    recs = df.to_dict(orient="records")

    # numpy 타입, Timestamp 등 처리
    def py_convert(v):
        if isinstance(v, (np.integer,)): return int(v)
        if isinstance(v, (np.floating,)) and not np.isnan(v): return float(v)
        if isinstance(v, (np.bool_,)): return bool(v)
        # Timestamp/Datetime -> ISO 문자열
        if isinstance(v, (pd.Timestamp,)): return v.strftime("%Y-%m-%dT%H:%M:%S") if pd.notna(v) else None
        return v

    def walk(o):
        if isinstance(o, dict):
            return {k: walk(py_convert(v)) for k, v in o.items()}
        if isinstance(o, list):
            return [walk(py_convert(x)) for x in o]
        return py_convert(o)

    return [walk(r) for r in recs]


# ========== 메인 ==========

def main():
    csv_files = [f for f in os.listdir(BASE_DIR) if f.lower().endswith(".csv")]
    if not csv_files:
        print("❌ CSV가 없습니다."); return

    for fn in csv_files:
        src = os.path.join(BASE_DIR, fn)
        stem = os.path.splitext(fn)[0]
        dst_json = os.path.join(OUT_DIR, f"{stem}.json")

        try:
            print(f"처리 중: {fn}")
            df = try_read_csv(src)
            df_clean = normalize_df(df)
            records = df_to_json_records(df_clean)

            with open(dst_json, "w", encoding="utf-8") as f:
                json.dump(records, f, ensure_ascii=False, indent=2)

            print(f"  ✓ 저장: {dst_json} (records: {len(records)})")
        except Exception as e:
            print(f"  ⚠️ 실패: {fn} -> {e}")

    print("\n✅ 완료: 모든 JSON은 clean/ 폴더에 저장되었습니다.")

if __name__ == "__main__":
    main()


처리 중: av_contents.csv
  ✓ 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\av_contents.json (records: 13)
처리 중: contents.csv
  ✓ 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\contents.json (records: 139)
처리 중: content_ratings.csv
  ✓ 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\content_ratings.json (records: 0)
처리 중: game_contents.csv
  ✓ 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\game_contents.json (records: 22)
처리 중: llm_recommendation_requests.csv


C:\Users\LG\AppData\Local\Temp\ipykernel_27168\4233549663.py:79: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace(COMMA_PLACEHOLDER, ",") if isinstance(x, str) else x)
C:\Users\LG\AppData\Local\Temp\ipykernel_27168\4233549663.py:79: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace(COMMA_PLACEHOLDER, ",") if isinstance(x, str) else x)
C:\Users\LG\AppData\Local\Temp\ipykernel_27168\4233549663.py:79: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace(COMMA_PLACEHOLDER, ",") if isinstance(x, str) else x)


  ✓ 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\llm_recommendation_requests.json (records: 0)
처리 중: naver_series_novel.csv
  ✓ 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\naver_series_novel.json (records: 0)
처리 중: platform_data.csv
  ✓ 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\platform_data.json (records: 123)
처리 중: raw_items.csv


C:\Users\LG\AppData\Local\Temp\ipykernel_27168\4233549663.py:79: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace(COMMA_PLACEHOLDER, ",") if isinstance(x, str) else x)
C:\Users\LG\AppData\Local\Temp\ipykernel_27168\4233549663.py:79: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace(COMMA_PLACEHOLDER, ",") if isinstance(x, str) else x)


  ✓ 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\raw_items.json (records: 6)
처리 중: transform_runs.csv
  ✓ 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\transform_runs.json (records: 250)
처리 중: users.csv
  ✓ 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\users.json (records: 0)
처리 중: user_preferences.csv
  ✓ 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\user_preferences.json (records: 0)
처리 중: user_preferred_content_types.csv
  ✓ 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\user_preferred_content_types.json (records: 0)
처리 중: user_preferred_genres.csv
  ✓ 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\user_preferred_genres.json (records: 0)
처리 중: user_roles.csv
  ✓ 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\user_roles.json (records: 0)
처리 중: webnovel_contents.csv
  ✓ 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\webnovel_contents.json (records: 93)
처리 중: webtoon_contents.csv
  ✓ 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\cle

# 이분 그래프 생성

In [3]:
# -*- coding: utf-8 -*-

import os
import re
import pandas as pd

BASE = r"csv 데이터\clean"
PATH_CONTENTS = os.path.join(BASE, "contents.csv")
PATH_AV       = os.path.join(BASE, "av_contents.csv")
PATH_GAME     = os.path.join(BASE, "game_contents.csv")
PATH_WEBNOVEL = os.path.join(BASE, "webnovel_contents.csv")  # 있으면 사용

OUT_NODES     = os.path.join(BASE, "graph_nodes.csv")              # 콘텐츠 노드
OUT_RAWGEN    = os.path.join(BASE, "content_raw_genres.csv")       # 원본 장르 세로표
OUT_META      = os.path.join(BASE, "meta_nodes.csv")               # 메타노드 목록
OUT_EDGES_BI  = os.path.join(BASE, "graph_edges_bipartite.csv")    # 콘텐츠→메타 엣지

# 엣지 가중치(필요 시 조정)
DOMAIN_EDGE_WEIGHT = 1.0
GENRE_EDGE_WEIGHT  = 1.0

ENCODINGS = ["utf-8-sig","utf-8","cp949","euc-kr","latin1"]
def read_csv_retry(path, **kwargs):
    last = None
    for enc in ENCODINGS:
        try:
            return pd.read_csv(path, encoding=enc, **kwargs)
        except Exception as e:
            last = e
    raise last

def safe_read(path):
    if not os.path.exists(path):
        print(f"⚠️ 없음: {path}")
        return None
    try:
        return read_csv_retry(path)
    except Exception as e:
        print(f"⚠️ 읽기 실패: {path} -> {e}")
        return None

def add_prefix_except_key(df, prefix, key="content_id"):
    if df is None: return None
    return df.rename(columns={c: (c if c==key else f"{prefix}{c}") for c in df.columns})

def norm_for_id(s: str) -> str:
    """
    메타노드 ID용 간단 정규화 (공백/구두점 제거, 소문자)
    예: 'Science Fiction' -> 'science_fiction'
    """
    if s is None: return ""
    s = str(s).strip().lower()
    # 한/영/숫자 외는 공백 치환
    s = re.sub(r"[^\w\s가-힣]", " ", s)
    s = "_".join(s.split())
    return s

# 1) 콘텐츠 노드 생성(그대로 유지)
def build_nodes():
    contents = safe_read(PATH_CONTENTS)
    if contents is None or contents.empty:
        raise RuntimeError("❌ contents.csv 비어있음/로드 실패")
    if "content_id" not in contents.columns or "domain" not in contents.columns:
        raise RuntimeError("❌ contents.csv에 content_id/domain 컬럼 필요")

    av   = add_prefix_except_key(safe_read(PATH_AV), "av_")
    game = add_prefix_except_key(safe_read(PATH_GAME), "game_")
    web  = add_prefix_except_key(safe_read(PATH_WEBNOVEL), "webnovel_")

    def typed_left_merge(left, right, key="content_id"):
        if right is None or right.empty:
            return left.copy()
        L = left.copy(); R = right.copy()
        if key in L.columns: L[key] = pd.to_numeric(L[key], errors="coerce").astype("Int64")
        if key in R.columns: R[key] = pd.to_numeric(R[key], errors="coerce").astype("Int64")
        return L.merge(R, on=key, how="left")

    parts = []
    doms = contents["domain"].dropna().unique().tolist()
    if "AV" in doms:
        sub = contents[contents["domain"]=="AV"].copy()
        sub = typed_left_merge(sub, av); parts.append(sub)
    if "GAME" in doms:
        sub = contents[contents["domain"]=="GAME"].copy()
        sub = typed_left_merge(sub, game); parts.append(sub)
    if "WEBNOVEL" in doms:
        sub = contents[contents["domain"]=="WEBNOVEL"].copy()
        sub = typed_left_merge(sub, web); parts.append(sub)

    known = {"AV","GAME","WEBNOVEL"}
    nodes = pd.concat(parts, ignore_index=True) if parts else contents[contents["domain"].isin(known)].copy()
    nodes = nodes.drop_duplicates(subset=["content_id"], keep="first")

    # 보기 좋게 정렬
    first = [c for c in ["content_id","domain","master_title","original_title","release_year",
                         "poster_image_url","created_at","updated_at","synopsis"] if c in nodes.columns]
    nodes = nodes[first + [c for c in nodes.columns if c not in first]]

    nodes.to_csv(OUT_NODES, index=False, encoding="utf-8-sig")
    print(f"✅ graph_nodes 저장: {OUT_NODES} (rows={len(nodes)})")
    return nodes

# 2) 원본 장르 세로표(content_id, source, raw_genre)
def build_raw_genres(nodes):
    valid_ids = set(pd.to_numeric(nodes["content_id"], errors="coerce").dropna().astype(int))
    rows = []

    # AV: TMDB 장르 (genres.tmdb_genres.N.name)
    av = safe_read(PATH_AV)
    if av is not None and not av.empty:
        name_cols = [c for c in av.columns if c.startswith("genres.tmdb_genres.") and c.endswith(".name")]
        if name_cols:
            tmp = av.melt(id_vars=["content_id"], value_vars=name_cols, value_name="raw_genre").dropna(subset=["raw_genre"])
            tmp["raw_genre"] = tmp["raw_genre"].astype(str).str.strip()
            tmp = tmp[(tmp["raw_genre"]!="") & (tmp["content_id"].isin(valid_ids))]
            tmp["source"] = "tmdb"
            rows.append(tmp[["content_id","source","raw_genre"]])

    # GAME: Steam 장르 (genres_str: "A;B;C")
    game = safe_read(PATH_GAME)
    if game is not None and not game.empty and "genres_str" in game.columns:
        g2 = game.dropna(subset=["genres_str"]).copy()
        g2["raw_genre"] = g2["genres_str"].astype(str).str.split(r"\s*;\s*")
        g2 = g2.explode("raw_genre").dropna(subset=["raw_genre"])
        g2["raw_genre"] = g2["raw_genre"].astype(str).str.strip()
        g2 = g2[(g2["raw_genre"]!="") & (g2["content_id"].isin(valid_ids))]
        g2["source"] = "steam"
        rows.append(g2[["content_id","source","raw_genre"]])

    # WEBNOVEL: 장르 컬럼이 있다면 분해 (예: 'genres'가 "판타지;로맨스")
    web = safe_read(PATH_WEBNOVEL)
    if web is not None and not web.empty:
        # 후보 컬럼들 중 존재하는 것 선택
        cand_cols = [c for c in ["genres","genre","genre_str"] if c in web.columns]
        if cand_cols:
            col = cand_cols[0]
            w2 = web.dropna(subset=[col]).copy()
            # 구분자: ; , / | (혼용 방지용 정규식)
            w2["raw_genre"] = w2[col].astype(str).str.split(r"\s*[;|/,]\s*")
            w2 = w2.explode("raw_genre").dropna(subset=["raw_genre"])
            w2["raw_genre"] = w2["raw_genre"].astype(str).str.strip()
            if "content_id" in w2.columns:
                w2 = w2[w2["raw_genre"]!=""]
                w2 = w2[w2["content_id"].isin(valid_ids)]
                w2["source"] = "webnovel"
                rows.append(w2[["content_id","source","raw_genre"]])

    df = pd.concat(rows, ignore_index=True) if rows else pd.DataFrame(columns=["content_id","source","raw_genre"])
    if not df.empty:
        df["content_id"] = pd.to_numeric(df["content_id"], errors="coerce").astype("Int64")
        df = df[df["content_id"].notna()]
        df["content_id"] = df["content_id"].astype(int)

    df = df.drop_duplicates().reset_index(drop=True)
    df.to_csv(OUT_RAWGEN, index=False, encoding="utf-8-sig")
    print(f"✅ content_raw_genres 저장: {OUT_RAWGEN} (rows={len(df)})")
    return df

# 3) 메타노드 + 이분 엣지 생성 (도메인/장르 모두 '원본' 기준)
def build_bipartite(nodes, raw_genres):
    # 메타노드: 도메인
    domain_nodes = sorted(nodes["domain"].dropna().unique().tolist())

    meta_rows = []
    for d in domain_nodes:
        meta_rows.append({"meta_id": f"DOM:{d}", "meta_type": "domain", "label": d, "source": ""})

    # 메타노드: 장르 (네임스페이스 적용)
    if raw_genres is not None and not raw_genres.empty:
        # (source, raw_genre_norm)별 하나의 메타노드
        rg = raw_genres.copy()
        rg["raw_genre_norm"] = rg["raw_genre"].astype(str).str.strip()
        rg["raw_genre_norm"] = rg["raw_genre_norm"].replace({"": None})
        rg = rg.dropna(subset=["raw_genre_norm"])
        uniq = rg[["source","raw_genre_norm"]].drop_duplicates()
        for src, g in uniq.itertuples(index=False):
            meta_id = f"GEN:{src}:{norm_for_id(g)}" if g else None
            if meta_id:
                meta_rows.append({"meta_id": meta_id, "meta_type": "genre", "label": g, "source": src})

    df_meta = pd.DataFrame(meta_rows, columns=["meta_id","meta_type","label","source"]).drop_duplicates()
    df_meta.to_csv(OUT_META, index=False, encoding="utf-8-sig")
    print(f"✅ meta_nodes 저장: {OUT_META} (rows={len(df_meta)})")

    # 이분 엣지: 콘텐츠 → 도메인
    edges = []
    sub = nodes[["content_id","domain"]].dropna()
    for cid, dom in sub.itertuples(index=False):
        edges.append({
            "src_content_id": int(cid),
            "dst_meta_id":    f"DOM:{dom}",
            "edge_type":      "content-domain",
            "weight":         DOMAIN_EDGE_WEIGHT
        })

    # 이분 엣지: 콘텐츠 → (source별 원본장르)
    if raw_genres is not None and not raw_genres.empty:
        for cid, src, g in raw_genres[["content_id","source","raw_genre"]].itertuples(index=False):
            g_norm = norm_for_id(g)
            if not g_norm:
                continue
            meta_id = f"GEN:{src}:{g_norm}"
            edges.append({
                "src_content_id": int(cid),
                "dst_meta_id":    meta_id,
                "edge_type":      f"content-genre-{src}",
                "weight":         GENRE_EDGE_WEIGHT
            })

    df_edges = pd.DataFrame(edges, columns=["src_content_id","dst_meta_id","edge_type","weight"]).drop_duplicates()
    df_edges.to_csv(OUT_EDGES_BI, index=False, encoding="utf-8-sig")
    print(f"✅ graph_edges_bipartite 저장: {OUT_EDGES_BI} (edges={len(df_edges)})")
    return df_meta, df_edges

def main():
    nodes = build_nodes()
    rawg  = build_raw_genres(nodes)
    build_bipartite(nodes, rawg)

if __name__ == "__main__":
    main()


✅ graph_nodes 저장: csv 데이터\clean\graph_nodes.csv (rows=495)
✅ content_raw_genres 저장: csv 데이터\clean\content_raw_genres.csv (rows=630)
✅ meta_nodes 저장: csv 데이터\clean\meta_nodes.csv (rows=45)
✅ graph_edges_bipartite 저장: csv 데이터\clean\graph_edges_bipartite.csv (edges=1125)


# 이분그래프 생성 1.0

In [4]:
# -*- coding: utf-8 -*-

import os
import re
import pandas as pd
from typing import Optional, List

BASE = r"csv 데이터\clean"
PATH_CONTENTS = os.path.join(BASE, "contents.csv")
PATH_AV       = os.path.join(BASE, "av_contents.csv")
PATH_GAME     = os.path.join(BASE, "game_contents.csv")
PATH_WEBNOVEL = os.path.join(BASE, "webnovel_contents.csv")  # 있으면 사용

PATH_RAW_ITEM = os.path.join(BASE, "raw_item.csv")           # raw_id, genres_str

OUT_NODES     = os.path.join(BASE, "graph_nodes.csv")              # 콘텐츠 노드
OUT_RAWGEN    = os.path.join(BASE, "content_raw_genres.csv")       # (최종) content_id, source, raw_genre_1~3
OUT_META      = os.path.join(BASE, "meta_nodes.csv")               # 메타노드 목록
OUT_EDGES_BI  = os.path.join(BASE, "graph_edges_bipartite.csv")    # 콘텐츠→메타 엣지

# 엣지 가중치(필요 시 조정)
DOMAIN_EDGE_WEIGHT = 1.0
GENRE_EDGE_WEIGHT  = 1.0

ENCODINGS = ["utf-8-sig","utf-8","cp949","euc-kr","latin1"]
def read_csv_retry(path, **kwargs):
    last = None
    for enc in ENCODINGS:
        try:
            return pd.read_csv(path, encoding=enc, **kwargs)
        except Exception as e:
            last = e
    raise last

def safe_read(path):
    if not os.path.exists(path):
        print(f"⚠️ 없음: {path}")
        return None
    try:
        return read_csv_retry(path)
    except Exception as e:
        print(f"⚠️ 읽기 실패: {path} -> {e}")
        return None

def add_prefix_except_key(df, prefix, key="content_id"):
    if df is None: return None
    return df.rename(columns={c: (c if c==key else f"{prefix}{c}") for c in df.columns})

def norm_for_id(s: str) -> str:
    """
    메타노드 ID용 간단 정규화 (공백/구두점 제거, 소문자)
    예: 'Science Fiction' -> 'science_fiction'
    """
    if s is None: return ""
    s = str(s).strip().lower()
    # 한/영/숫자 외는 공백 치환
    s = re.sub(r"[^\w\s가-힣]", " ", s)
    s = "_".join(s.split())
    return s

def parse_genres(genres_str: str) -> List[str]:
    """
    '드라마|범죄|미스터리' → ['드라마','범죄','미스터리']
    공백/콤마/슬래시/· 같은 것도 구분자로 처리
    """
    if not isinstance(genres_str, str):
        return []
    genres_str = genres_str.strip()
    if not genres_str:
        return []
    parts = re.split(r"[|,/·;]", genres_str)
    parts = [g.strip() for g in parts if g.strip()]
    return parts

# 1) 콘텐츠 노드 생성(그대로 유지)
def build_nodes():
    contents = safe_read(PATH_CONTENTS)
    if contents is None or contents.empty:
        raise RuntimeError("❌ contents.csv 비어있음/로드 실패")
    if "content_id" not in contents.columns or "domain" not in contents.columns:
        raise RuntimeError("❌ contents.csv에 content_id/domain 컬럼 필요")

    av   = add_prefix_except_key(safe_read(PATH_AV), "av_")
    game = add_prefix_except_key(safe_read(PATH_GAME), "game_")
    web  = add_prefix_except_key(safe_read(PATH_WEBNOVEL), "webnovel_")

    def typed_left_merge(left, right, key="content_id"):
        if right is None or right.empty:
            return left.copy()
        L = left.copy(); R = right.copy()
        if key in L.columns: L[key] = pd.to_numeric(L[key], errors="coerce").astype("Int64")
        if key in R.columns: R[key] = pd.to_numeric(R[key], errors="coerce").astype("Int64")
        return L.merge(R, on=key, how="left")

    parts = []
    doms = contents["domain"].dropna().unique().tolist()
    if "AV" in doms:
        sub = contents[contents["domain"]=="AV"].copy()
        sub = typed_left_merge(sub, av); parts.append(sub)
    if "GAME" in doms:
        sub = contents[contents["domain"]=="GAME"].copy()
        sub = typed_left_merge(sub, game); parts.append(sub)
    if "WEBNOVEL" in doms:
        sub = contents[contents["domain"]=="WEBNOVEL"].copy()
        sub = typed_left_merge(sub, web); parts.append(sub)

    known = {"AV","GAME","WEBNOVEL"}
    nodes = pd.concat(parts, ignore_index=True) if parts else contents[contents["domain"].isin(known)].copy()
    nodes = nodes.drop_duplicates(subset=["content_id"], keep="first")

    # 보기 좋게 정렬
    first = [c for c in ["content_id","domain","master_title","original_title","release_year",
                         "poster_image_url","created_at","updated_at","synopsis"] if c in nodes.columns]
    nodes = nodes[first + [c for c in nodes.columns if c not in first]]

    nodes.to_csv(OUT_NODES, index=False, encoding="utf-8-sig")
    print(f"✅ graph_nodes 저장: {OUT_NODES} (rows={len(nodes)})")
    return nodes

# 2) 장르 표 생성
#   2-1) 우선 예전처럼 세로형(content_id, source, raw_genre)을 만들고
#   2-2) raw_item.csv의 genres_str를 이용해 최대 3개 장르로 가로형(raw_genre_1~3)으로 변환
def build_raw_genres(nodes):
    valid_ids = set(pd.to_numeric(nodes["content_id"], errors="coerce").dropna().astype(int))
    rows = []

    # ----- AV: TMDB 장르 (genres.tmdb_genres.N.name) -----
    av = safe_read(PATH_AV)
    if av is not None and not av.empty:
        av = av.copy()
        av.columns = [c.strip() for c in av.columns]
        name_cols = [c for c in av.columns if c.startswith("genres.tmdb_genres.") and c.endswith(".name")]
        if name_cols and "content_id" in av.columns:
            tmp = av.melt(id_vars=["content_id"], value_vars=name_cols, value_name="raw_genre").dropna(subset=["raw_genre"])
            tmp["raw_genre"] = tmp["raw_genre"].astype(str).str.strip()
            tmp = tmp[(tmp["raw_genre"]!="") & (tmp["content_id"].isin(valid_ids))]
            tmp["source"] = "tmdb"
            rows.append(tmp[["content_id","source","raw_genre"]])

    # ----- GAME: Steam 장르 (genres_str: "A;B;C") -----
    game = safe_read(PATH_GAME)
    if game is not None and not game.empty:
        game = game.copy()
        game.columns = [c.strip() for c in game.columns]
        if "genres_str" in game.columns and "content_id" in game.columns:
            g2 = game.dropna(subset=["genres_str"]).copy()
            g2["raw_genre"] = g2["genres_str"].astype(str).str.split(r"\s*;\s*")
            g2 = g2.explode("raw_genre").dropna(subset=["raw_genre"])
            g2["raw_genre"] = g2["raw_genre"].astype(str).str.strip()
            g2 = g2[(g2["raw_genre"]!="") & (g2["content_id"].isin(valid_ids))]
            g2["source"] = "steam"
            rows.append(g2[["content_id","source","raw_genre"]])

    # ----- WEBNOVEL: 장르 컬럼이 있다면 분해 (예: 'genres'가 "판타지;로맨스") -----
    web = safe_read(PATH_WEBNOVEL)
    if web is not None and not web.empty:
        web = web.copy()
        web.columns = [c.strip() for c in web.columns]
        cand_cols = [c for c in ["genres","genre","genre_str"] if c in web.columns]
        if cand_cols:
            col = cand_cols[0]
            w2 = web.dropna(subset=[col]).copy()
            w2["raw_genre"] = w2[col].astype(str).str.split(r"\s*[;|/,]\s*")
            w2 = w2.explode("raw_genre").dropna(subset=["raw_genre"])
            w2["raw_genre"] = w2["raw_genre"].astype(str).str.strip()
            if "content_id" in w2.columns:
                w2 = w2[w2["raw_genre"]!=""]
                w2 = w2[w2["content_id"].isin(valid_ids)]
                w2["source"] = "webnovel"
                rows.append(w2[["content_id","source","raw_genre"]])

    # ----- 2-1) tall: content_id, source, raw_genre -----
    if rows:
        cg = pd.concat(rows, ignore_index=True)
        cg["content_id"] = pd.to_numeric(cg["content_id"], errors="coerce").astype("Int64")
        cg = cg[cg["content_id"].notna()]
        cg["content_id"] = cg["content_id"].astype(int)
        cg["raw_genre"] = cg["raw_genre"].astype(str).str.strip()
        cg = cg[cg["raw_genre"]!=""]
        cg = cg.drop_duplicates().reset_index(drop=True)
    else:
        cg = pd.DataFrame(columns=["content_id","source","raw_genre"])

    # ----- 2-2) raw_item.csv 의 genres_str 를 이용해 raw_genre_1~3 생성 -----
    ri = safe_read(PATH_RAW_ITEM)
    if ri is not None and not ri.empty:
        ri = ri.copy()
        ri.columns = [c.strip() for c in ri.columns]

        needed_ri = ["raw_id", "genres_str"]
        miss_ri = [c for c in needed_ri if c not in ri.columns]
        if miss_ri:
            print("⚠️ raw_item.csv 에 다음 컬럼이 없어 genres_str를 사용할 수 없습니다:", ", ".join(miss_ri))
            ri = None
        else:
            ri["raw_id"] = pd.to_numeric(ri["raw_id"], errors="coerce").astype("Int64")
            ri = ri[ri["raw_id"].notna()].copy()
            ri["raw_id"] = ri["raw_id"].astype(int)
    else:
        ri = None

    if ri is not None and not cg.empty:
        merged = cg.merge(
            ri[["raw_id", "genres_str"]],
            left_on="content_id",
            right_on="raw_id",
            how="left"
        )
    else:
        merged = cg.copy()
        merged["genres_str"] = ""

    # raw_genre_1,2,3 생성
    new_rows = []
    for _, row in merged.iterrows():
        cid    = int(row["content_id"])
        source = str(row["source"]).strip()
        base_genre = str(row["raw_genre"]).strip()

        genres_from_str = parse_genres(row.get("genres_str", ""))

        genres = list(genres_from_str)
        # 기존 raw_genre가 있고 리스트에 없으면 맨 앞에 추가
        if base_genre and base_genre not in genres:
            genres.insert(0, base_genre)

        g1 = genres[0] if len(genres) > 0 else ""
        g2 = genres[1] if len(genres) > 1 else ""
        g3 = genres[2] if len(genres) > 2 else ""

        new_rows.append((cid, source, g1, g2, g3))

    wide = pd.DataFrame(new_rows, columns=["content_id","source","raw_genre_1","raw_genre_2","raw_genre_3"])
    wide = wide.drop_duplicates(subset=["content_id","source","raw_genre_1","raw_genre_2","raw_genre_3"])

    wide.to_csv(OUT_RAWGEN, index=False, encoding="utf-8-sig")
    print(f"✅ content_raw_genres (wide) 저장: {OUT_RAWGEN} (rows={len(wide)})")
    return wide

# wide(raw_genre_1~3) → tall(raw_genre) 변환 (그래프 내부용)
def to_tall_raw_genres(raw_genres: pd.DataFrame) -> pd.DataFrame:
    df = raw_genres.copy()
    df.columns = [c.strip() for c in df.columns]

    # 이미 raw_genre 단일 컬럼이면 그대로 사용
    if "raw_genre" in df.columns and not any(
        c in df.columns for c in ["raw_genre_1","raw_genre_2","raw_genre_3"]
    ):
        df["content_id"] = pd.to_numeric(df["content_id"], errors="coerce").astype("Int64")
        df = df[df["content_id"].notna()].copy()
        df["content_id"] = df["content_id"].astype(int)
        df["raw_genre"] = df["raw_genre"].astype(str).str.strip()
        df = df[df["raw_genre"]!=""]
        return df[["content_id","source","raw_genre"]].drop_duplicates().reset_index(drop=True)

    # 가로형 raw_genre_1~3 → 세로형
    genre_cols = [c for c in ["raw_genre_1","raw_genre_2","raw_genre_3"] if c in df.columns]
    if not genre_cols:
        raise RuntimeError("raw_genres에 raw_genre 또는 raw_genre_1/2/3 컬럼이 없습니다.")

    df["content_id"] = pd.to_numeric(df["content_id"], errors="coerce").astype("Int64")
    df = df[df["content_id"].notna()].copy()
    df["content_id"] = df["content_id"].astype(int)

    tall = df.melt(
        id_vars=["content_id","source"],
        value_vars=genre_cols,
        value_name="raw_genre"
    )
    tall["raw_genre"] = tall["raw_genre"].astype(str).str.strip()
    tall = tall[tall["raw_genre"].notna() & (tall["raw_genre"]!="")]
    tall = tall[["content_id","source","raw_genre"]].drop_duplicates()

    print(f"✅ raw_genre_1~3 → 세로형 변환: rows={len(tall)}")
    return tall.reset_index(drop=True)

# 3) 메타노드 + 이분 엣지 생성
def build_bipartite(nodes, raw_genres):
    # 먼저 wide(raw_genre_1~3)를 tall(raw_genre)로 변환
    tall = to_tall_raw_genres(raw_genres) if raw_genres is not None and not raw_genres.empty else None

    # 메타노드: 도메인
    domain_nodes = sorted(nodes["domain"].dropna().unique().tolist())

    meta_rows = []
    for d in domain_nodes:
        meta_rows.append({"meta_id": f"DOM:{d}", "meta_type": "domain", "label": d, "source": ""})

    # 메타노드: 장르 (네임스페이스 적용)
    if tall is not None and not tall.empty:
        rg = tall.copy()
        rg["raw_genre_norm"] = rg["raw_genre"].astype(str).str.strip()
        rg["raw_genre_norm"] = rg["raw_genre_norm"].replace({"": None})
        rg = rg.dropna(subset=["raw_genre_norm"])
        uniq = rg[["source","raw_genre_norm"]].drop_duplicates()
        for src, g in uniq.itertuples(index=False):
            meta_id = f"GEN:{src}:{norm_for_id(g)}" if g else None
            if meta_id:
                meta_rows.append({"meta_id": meta_id, "meta_type": "genre", "label": g, "source": src})

    df_meta = pd.DataFrame(meta_rows, columns=["meta_id","meta_type","label","source"]).drop_duplicates()
    df_meta.to_csv(OUT_META, index=False, encoding="utf-8-sig")
    print(f"✅ meta_nodes 저장: {OUT_META} (rows={len(df_meta)})")

    # 이분 엣지: 콘텐츠 → 도메인
    edges = []
    sub = nodes[["content_id","domain"]].dropna()
    for cid, dom in sub.itertuples(index=False):
        edges.append({
            "src_content_id": int(cid),
            "dst_meta_id":    f"DOM:{dom}",
            "edge_type":      "content-domain",
            "weight":         DOMAIN_EDGE_WEIGHT
        })

    # 이분 엣지: 콘텐츠 → 장르 (한 콘텐츠가 여러 장르와 연결됨)
    if tall is not None and not tall.empty:
        for cid, src, g in tall[["content_id","source","raw_genre"]].itertuples(index=False):
            g_norm = norm_for_id(g)
            if not g_norm:
                continue
            meta_id = f"GEN:{src}:{g_norm}"
            edges.append({
                "src_content_id": int(cid),
                "dst_meta_id":    meta_id,
                "edge_type":      f"content-genre-{src}",
                "weight":         GENRE_EDGE_WEIGHT
            })

    df_edges = pd.DataFrame(edges, columns=["src_content_id","dst_meta_id","edge_type","weight"]).drop_duplicates()
    df_edges.to_csv(OUT_EDGES_BI, index=False, encoding="utf-8-sig")
    print(f"✅ graph_edges_bipartite 저장: {OUT_EDGES_BI} (edges={len(df_edges)})")
    return df_meta, df_edges

def main():
    nodes = build_nodes()
    rawg  = build_raw_genres(nodes)  # 여기서 content_raw_genres.csv (raw_genre_1~3) 생성
    build_bipartite(nodes, rawg)    # 여기서 한 콘텐츠가 여러 장르 노드와 엣지로 연결됨

if __name__ == "__main__":
    main()


✅ graph_nodes 저장: csv 데이터\clean\graph_nodes.csv (rows=495)
✅ content_raw_genres (wide) 저장: csv 데이터\clean\content_raw_genres.csv (rows=300)
✅ raw_genre_1~3 → 세로형 변환: rows=646
✅ meta_nodes 저장: csv 데이터\clean\meta_nodes.csv (rows=48)
✅ graph_edges_bipartite 저장: csv 데이터\clean\graph_edges_bipartite.csv (edges=1141)


# 아이템-아이템 그래프 생성

In [5]:
# -*- coding: utf-8 -*-


import os
import math
import pandas as pd
from collections import defaultdict

BASE = r"csv 데이터\clean"
PATH_NODES   = os.path.join(BASE, "graph_nodes.csv")
PATH_RAWGEN  = os.path.join(BASE, "content_raw_genres.csv")
OUT_EDGES    = os.path.join(BASE, "graph_edges_item_item.csv")

# ========= 하이퍼파라미터(조절 포인트) =========
DOMAIN_BASE_WEIGHT    = 1.0   # 같은 도메인 공유 기여치
GENRE_BASE_WEIGHT     = 1.0   # 같은 '도메인별 원본장르' 공유 기여치
ALPHA                 = 0.7   # 허브 완화 지수(0=완화 없음, 1=강한 완화). 보통 0.5~0.8
USE_IDF               = True  # 메타노드 정보량 가중치(log(1+N/deg)) 사용
TOPK_PER_ITEM         = 100   # 아이템당 상위 K 이웃만 유지 (희소화)
MAX_MEMBERS_PER_META  = 2000  # 메타노드(도메인/장르) 멤버가 너무 클 때 샘플링 상한 (메모리/시간 보호)

ENCODINGS = ["utf-8-sig","utf-8","cp949","euc-kr","latin1"]
def read_csv_retry(path, **kwargs):
    last = None
    for enc in ENCODINGS:
        try:
            return pd.read_csv(path, encoding=enc, **kwargs)
        except Exception as e:
            last = e
    raise last

def main():
    # 1) 로드
    nodes = read_csv_retry(PATH_NODES)
    rawg  = read_csv_retry(PATH_RAWGEN) if os.path.exists(PATH_RAWGEN) else pd.DataFrame(columns=["content_id","source","raw_genre"])

    if nodes.empty:
        raise RuntimeError("graph_nodes.csv 비어있음")
    if "content_id" not in nodes.columns or "domain" not in nodes.columns:
        raise RuntimeError("graph_nodes.csv에 content_id/domain 필요")

    nodes = nodes.dropna(subset=["content_id","domain"]).copy()
    nodes["content_id"] = pd.to_numeric(nodes["content_id"], errors="coerce").astype("Int64")
    nodes = nodes[nodes["content_id"].notna()]
    nodes["content_id"] = nodes["content_id"].astype(int)

    # 도메인 맵
    domain_of = dict(nodes[["content_id","domain"]].values)
    all_items = set(domain_of.keys())
    N_items   = max(1, len(all_items))

    # 2) 메타노드 멤버 구성
    # (a) 도메인 메타노드: DOM:<domain>
    members_of_meta = defaultdict(list)
    for cid, dom in domain_of.items():
        members_of_meta[f"DOM:{dom}"].append(cid)

    # (b) '도메인별 원본 장르' 메타노드: GEN:<source>:<raw_genre>
    if not rawg.empty:
        rawg = rawg.dropna(subset=["content_id","source","raw_genre"]).copy()
        rawg["content_id"] = pd.to_numeric(rawg["content_id"], errors="coerce").astype("Int64")
        rawg = rawg[rawg["content_id"].notna()]
        rawg["content_id"] = rawg["content_id"].astype(int)
        # 같은 도메인에서만 의미를 주려면, 아이템의 도메인을 확인해서 묶기만 하면 됨.
        # 여기선 메타노드 ID에 도메인까지 넣어 '도메인별 장르'로 분리한다.
        # 예) GEN:tmdb:공포@AV, GEN:steam:Action@GAME
        for cid, src, g in rawg[["content_id","source","raw_genre"]].drop_duplicates().itertuples(index=False):
            dom = domain_of.get(cid)
            if not dom: 
                continue
            meta_id = f"GEN:{src}:{g}@{dom}"
            members_of_meta[meta_id].append(cid)

    # 3) 투영(아이템↔아이템): 메타노드 공동소속 기반 가중치
    def meta_contrib(meta_id, deg):
        # deg = 메타노드 멤버 수
        base = DOMAIN_BASE_WEIGHT if meta_id.startswith("DOM:") else GENRE_BASE_WEIGHT
        if deg <= 1:
            return 0.0
        # 허브 완화: 1 / (deg^ALPHA)
        w = base / (deg ** ALPHA)
        # 정보량(IDF) 보정: log(1 + N / deg)
        if USE_IDF:
            w *= math.log(1.0 + (N_items / float(deg)))
        return w

    edges = defaultdict(float)

    for meta_id, members in members_of_meta.items():
        members = list(set(members))
        deg = len(members)
        if deg <= 1:
            continue

        # 너무 큰 메타노드(예: 특정 도메인 전체)가 폭발하지 않게 샘플링/상한
        if deg > MAX_MEMBERS_PER_META:
            # 균일 샘플링 (재현성은 보장X. 필요시 random.seed() 추가)
            members = members[:MAX_MEMBERS_PER_META]
            deg = len(members)

        contrib = meta_contrib(meta_id, deg)
        if contrib == 0.0:
            continue

        # 근린 제한 방식: 각 멤버마다 앞쪽 K개와만 연결 (전쌍 O(n^2) 방지)
        K = min(TOPK_PER_ITEM, deg - 1)
        members.sort()
        for i, a in enumerate(members):
            # 도메인 다른 쌍은 제외 (도메인 일치 강제) — 도메인별 장르 메타는 이미 분리되어 있지만,
            # 도메인 메타에서도 안전하게 동일 도메인만 유지
            for b in members[i+1 : i+1+K]:
                if domain_of.get(a) != domain_of.get(b):
                    continue
                if a > b:
                    a, b = b, a
                edges[(a,b)] += contrib

    # 4) 아이템당 TOPK 이웃만 유지(양방향 기준 최대 가중치 유지)
    nbrs = defaultdict(list)
    for (a,b), w in edges.items():
        nbrs[a].append((b,w))
        nbrs[b].append((a,w))

    pruned = {}
    for a, lst in nbrs.items():
        lst.sort(key=lambda x: x[1], reverse=True)
        for b, w in lst[:TOPK_PER_ITEM]:
            key = (a,b) if a < b else (b,a)
            if key not in pruned or pruned[key] < w:
                pruned[key] = w

    df_edges = pd.DataFrame(
        [(a,b,w) for (a,b), w in pruned.items()],
        columns=["src_content_id","dst_content_id","weight"]
    )
    df_edges.to_csv(OUT_EDGES, index=False, encoding="utf-8-sig")
    print(f"✅ 저장: {OUT_EDGES} (edges={len(df_edges):,}, items={len(all_items):,})")

if __name__ == "__main__":
    main()


KeyError: ['raw_genre']

# 아이템 그래프 생성1.0

In [6]:
# -*- coding: utf-8 -*-

import os
import math
import pandas as pd
from collections import defaultdict

BASE = r"csv 데이터\clean"
PATH_NODES   = os.path.join(BASE, "graph_nodes.csv")
PATH_RAWGEN  = os.path.join(BASE, "content_raw_genres.csv")
OUT_EDGES    = os.path.join(BASE, "graph_edges_item_item.csv")

# ========= 하이퍼파라미터(조절 포인트) =========
DOMAIN_BASE_WEIGHT    = 1.0   # 같은 도메인 공유 기여치
GENRE_BASE_WEIGHT     = 1.0   # 같은 '도메인별 원본장르' 공유 기여치
ALPHA                 = 0.7   # 허브 완화 지수(0=완화 없음, 1=강한 완화). 보통 0.5~0.8
USE_IDF               = True  # 메타노드 정보량 가중치(log(1+N/deg)) 사용
TOPK_PER_ITEM         = 100   # 아이템당 상위 K 이웃만 유지 (희소화)
MAX_MEMBERS_PER_META  = 2000  # 메타노드 멤버가 너무 클 때 샘플링 상한 (메모리/시간 보호)

ENCODINGS = ["utf-8-sig","utf-8","cp949","euc-kr","latin1"]

def read_csv_retry(path, **kwargs):
    last = None
    for enc in ENCODINGS:
        try:
            return pd.read_csv(path, encoding=enc, **kwargs)
        except Exception as e:
            last = e
    raise last

def to_tall_raw_genres(rawg: pd.DataFrame) -> pd.DataFrame:
    """
    content_raw_genres.csv가
      - (A) 세로형: content_id, source, raw_genre
      - (B) 가로형: content_id, source, raw_genre_1~3
    둘 중 어느 포맷이든 받을 수 있게 하고,
    최종적으로 항상 세로형 (content_id, source, raw_genre) 로 변환.
    """
    df = rawg.copy()
    df.columns = [c.strip() for c in df.columns]

    # (A) 이미 raw_genre 단일 컬럼이 있으면 그대로 정리해서 반환
    if "raw_genre" in df.columns:
        if "content_id" not in df.columns or "source" not in df.columns:
            raise RuntimeError("raw_genres에 content_id/source/raw_genre 컬럼이 모두 필요합니다.")
        df["content_id"] = pd.to_numeric(df["content_id"], errors="coerce").astype("Int64")
        df = df[df["content_id"].notna()].copy()
        df["content_id"] = df["content_id"].astype(int)
        df["raw_genre"] = df["raw_genre"].astype(str).str.strip()
        df = df[df["raw_genre"] != ""]
        return df[["content_id", "source", "raw_genre"]].drop_duplicates().reset_index(drop=True)

    # (B) 가로형(raw_genre_1~3)을 세로형으로 변환
    genre_cols = [c for c in ["raw_genre_1", "raw_genre_2", "raw_genre_3"] if c in df.columns]
    if not genre_cols:
        raise RuntimeError("content_raw_genres.csv에 raw_genre 또는 raw_genre_1/2/3 컬럼이 없습니다.")

    if "content_id" not in df.columns or "source" not in df.columns:
        raise RuntimeError("content_raw_genres.csv에 content_id/source 컬럼이 필요합니다.")

    df["content_id"] = pd.to_numeric(df["content_id"], errors="coerce").astype("Int64")
    df = df[df["content_id"].notna()].copy()
    df["content_id"] = df["content_id"].astype(int)

    tall = df.melt(
        id_vars=["content_id", "source"],
        value_vars=genre_cols,
        value_name="raw_genre"
    )
    tall["raw_genre"] = tall["raw_genre"].astype(str).str.strip()
    tall = tall[tall["raw_genre"].notna() & (tall["raw_genre"] != "")]
    tall = tall[["content_id", "source", "raw_genre"]].drop_duplicates()

    print(f"✅ raw_genre_1~3 → 세로형 변환: rows={len(tall)}")
    return tall.reset_index(drop=True)

def main():
    # 1) 노드 로드
    nodes = read_csv_retry(PATH_NODES)
    if nodes is None or nodes.empty:
        raise RuntimeError("graph_nodes.csv 비어있음/로드 실패")
    if "content_id" not in nodes.columns or "domain" not in nodes.columns:
        raise RuntimeError("graph_nodes.csv에 content_id/domain 컬럼 필요")

    nodes = nodes.dropna(subset=["content_id", "domain"]).copy()
    nodes["content_id"] = pd.to_numeric(nodes["content_id"], errors="coerce").astype("Int64")
    nodes = nodes[nodes["content_id"].notna()]
    nodes["content_id"] = nodes["content_id"].astype(int)

    # 도메인 맵
    domain_of = dict(nodes[["content_id", "domain"]].values)
    all_items = set(domain_of.keys())
    N_items   = max(1, len(all_items))

    # 2) content_raw_genres 로드 (wide → tall 변환 포함)
    if os.path.exists(PATH_RAWGEN):
        rawg_raw = read_csv_retry(PATH_RAWGEN)
        if rawg_raw is None or rawg_raw.empty:
            rawg = pd.DataFrame(columns=["content_id", "source", "raw_genre"])
        else:
            rawg = to_tall_raw_genres(rawg_raw)
    else:
        print("⚠️ content_raw_genres.csv 없음 → 장르 기반 기여 없이 도메인만 사용")
        rawg = pd.DataFrame(columns=["content_id", "source", "raw_genre"])

    # 3) 메타노드 멤버 구성
    members_of_meta = defaultdict(list)

    # (a) 도메인 메타노드: DOM:<domain>
    for cid, dom in domain_of.items():
        members_of_meta[f"DOM:{dom}"].append(cid)

    # (b) 도메인별 장르 메타노드: GEN:<source>:<raw_genre>@<domain>
    if not rawg.empty:
        rawg = rawg.dropna(subset=["content_id", "source", "raw_genre"]).copy()
        rawg["content_id"] = pd.to_numeric(rawg["content_id"], errors="coerce").astype("Int64")
        rawg = rawg[rawg["content_id"].notna()]
        rawg["content_id"] = rawg["content_id"].astype(int)

        for cid, src, g in rawg[["content_id", "source", "raw_genre"]].drop_duplicates().itertuples(index=False):
            dom = domain_of.get(cid)
            if not dom:
                continue
            meta_id = f"GEN:{src}:{g}@{dom}"
            members_of_meta[meta_id].append(cid)

    # 4) 메타노드 공동소속 기반 아이템↔아이템 가중치 계산
    def meta_contrib(meta_id, deg):
        # deg = 메타노드 멤버 수
        base = DOMAIN_BASE_WEIGHT if meta_id.startswith("DOM:") else GENRE_BASE_WEIGHT
        if deg <= 1:
            return 0.0
        # 허브 완화: 1 / (deg^ALPHA)
        w = base / (deg ** ALPHA)
        # 정보량(IDF) 보정: log(1 + N / deg)
        if USE_IDF:
            w *= math.log(1.0 + (N_items / float(deg)))
        return w

    edges = defaultdict(float)

    for meta_id, members in members_of_meta.items():
        members = list(set(members))
        deg = len(members)
        if deg <= 1:
            continue

        # 너무 큰 메타노드(예: 특정 도메인 전체)가 폭발하지 않게 샘플링/상한
        if deg > MAX_MEMBERS_PER_META:
            members = members[:MAX_MEMBERS_PER_META]
            deg = len(members)

        contrib = meta_contrib(meta_id, deg)
        if contrib == 0.0:
            continue

        # 근린 제한: 각 멤버마다 앞쪽 K개와만 연결 (전쌍 O(n^2) 방지)
        K = min(TOPK_PER_ITEM, deg - 1)
        members.sort()
        for i, a in enumerate(members):
            for b in members[i+1 : i+1+K]:
                # 서로 도메인이 다르면 스킵 (안전장치)
                if domain_of.get(a) != domain_of.get(b):
                    continue
                if a > b:
                    a, b = b, a
                edges[(a, b)] += contrib

    # 5) 아이템당 TOPK 이웃만 유지 (양방향 기준 최대 가중치 유지)
    nbrs = defaultdict(list)
    for (a, b), w in edges.items():
        nbrs[a].append((b, w))
        nbrs[b].append((a, w))

    pruned = {}
    for a, lst in nbrs.items():
        lst.sort(key=lambda x: x[1], reverse=True)
        for b, w in lst[:TOPK_PER_ITEM]:
            key = (a, b) if a < b else (b, a)
            if key not in pruned or pruned[key] < w:
                pruned[key] = w

    df_edges = pd.DataFrame(
        [(a, b, w) for (a, b), w in pruned.items()],
        columns=["src_content_id", "dst_content_id", "weight"]
    )
    df_edges.to_csv(OUT_EDGES, index=False, encoding="utf-8-sig")
    print(f"✅ 저장: {OUT_EDGES} (edges={len(df_edges):,}, items={len(all_items):,})")

if __name__ == "__main__":
    main()


✅ raw_genre_1~3 → 세로형 변환: rows=801
✅ 저장: csv 데이터\clean\graph_edges_item_item.csv (edges=31,148, items=495)


# Noce2vec 아이템 임베딩

In [7]:
# learn_item_embeddings.py
# -*- coding: utf-8 -*-

#아이템-아이템 가중 그래프 -> Node2Vec 스타일 임베딩 (Pure PyTorch, gensim/scipy 불필요)
#입력 : C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\graph_edges_item_item.csv
#출력 : C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\item_embeddings_torch.csv
#사용 : python learn_item_embeddings.py
#필요 : pip install torch pandas numpy
import os, math, random
import numpy as np
import pandas as pd
import torch
from collections import defaultdict

# ===== 경로 =====
BASE = r"csv 데이터\clean"
EDGES_PATH = os.path.join(BASE, "graph_edges_item_item.csv")
OUT_EMB_CSV = os.path.join(BASE, "item_embeddings_torch.csv")

# ===== 하이퍼파라미터 =====
DIM            = 64       # 임베딩 차원
WALK_LENGTH    = 40       # 랜덤워크 길이
NUM_WALKS      = 10       # 노드당 워크 수
P_RETURN       = 1.0      # node2vec p (되돌아가기 선호 p↓)
Q_INOUT        = 1.0      # node2vec q (원거리 탐색 선호 q↑)
WINDOW         = 5        # Skip-gram 윈도우
NEGATIVE_K     = 5        # 네거티브 샘플 수
EPOCHS         = 3
BATCH_SIZE     = 8192
LR             = 0.025
SEED           = 42
DEVICE         = "cuda" if torch.cuda.is_available() else "cpu"

random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

# ---------- 그래프 로드 & 가중 룰렛 준비 ----------
def load_graph(edges_csv):
    df = pd.read_csv(edges_csv, encoding="utf-8-sig")
    need = {"src_content_id","dst_content_id","weight"}
    if not need.issubset(df.columns):
        raise ValueError(f"{edges_csv}에 {need} 컬럼이 필요합니다.")

    adj = defaultdict(list)   # node -> [(nbr, w), ...]
    nodes = set()
    for a, b, w in df[["src_content_id","dst_content_id","weight"]].itertuples(index=False):
        if a == b: 
            continue
        w = float(w) if pd.notna(w) else 1.0
        adj[a].append((b, w))
        adj[b].append((a, w))
        nodes.add(a); nodes.add(b)

    # 누적확률(룰렛휠) 사전
    cumsums = {}
    for u, lst in adj.items():
        if not lst:
            continue
        nbrs, ws = zip(*lst)
        ws = np.maximum(np.array(ws, dtype=float), 1e-12)
        csum = np.cumsum(ws); cumsums[u] = (np.array(nbrs, dtype=int), csum / csum[-1])
    return adj, cumsums, sorted(nodes)

def weighted_choice(nbrs, cprobs):
    r = random.random()
    lo, hi = 0, len(cprobs)-1
    while lo < hi:
        mid = (lo + hi) // 2
        if cprobs[mid] < r: lo = mid + 1
        else: hi = mid
    return int(nbrs[lo])

# ---------- Node2Vec 가중 랜덤워크 ----------
def node2vec_walks(adj, cumsums, nodes, walk_length, num_walks, p, q):
    walks = []
    base_nodes = list(nodes)
    for _ in range(num_walks):
        random.shuffle(base_nodes)
        for start in base_nodes:
            if start not in cumsums:
                continue
            walk = [start]
            if walk_length == 1:
                walks.append(walk); continue

            # 첫 스텝: 가중치 비례 선택
            nbrs, cprobs = cumsums[start]
            if len(nbrs) == 0:
                walks.append(walk); continue
            curr = weighted_choice(nbrs, cprobs)
            walk.append(curr); prev = start

            for _ in range(2, walk_length):
                cand = adj.get(curr, [])
                if not cand: break
                cand_nodes = [n for n,_ in cand]
                cand_w = []
                prev_nbrs = {n for n,_ in adj.get(prev, [])}
                for nxt, w in cand:
                    # p/q 바이어스(근사)
                    if nxt == prev: bias = 1.0 / p
                    elif nxt in prev_nbrs: bias = 1.0
                    else: bias = 1.0 / q
                    cand_w.append(max(w,1e-12) * bias)
                cw = np.array(cand_w, dtype=float)
                cs = np.cumsum(cw); cs /= cs[-1]
                idx = np.searchsorted(cs, random.random())
                nxt = cand_nodes[min(idx, len(cand_nodes)-1)]
                walk.append(nxt); prev, curr = curr, nxt
            walks.append(walk)
    return walks

# ---------- Skip-gram(네거티브 샘플링) ----------
def generate_pairs(walks, window):
    pairs = []
    for walk in walks:
        L = len(walk)
        for i in range(L):
            c = walk[i]
            l = max(0, i-window); r = min(L, i+window+1)
            for j in range(l, r):
                if j == i: continue
                pairs.append((c, walk[j]))
    return pairs

class SkipGramNS(torch.nn.Module):
    def __init__(self, vocab_size, dim):
        super().__init__()
        self.in_emb  = torch.nn.Embedding(vocab_size, dim)
        self.out_emb = torch.nn.Embedding(vocab_size, dim)
        torch.nn.init.uniform_(self.in_emb.weight,  -0.5/dim, 0.5/dim)
        torch.nn.init.uniform_(self.out_emb.weight, -0.5/dim, 0.5/dim)

    def forward(self, center, pos, neg):
        # center:[B], pos:[B], neg:[B,K]
        v  = self.in_emb(center)           # [B, D]
        u  = self.out_emb(pos)             # [B, D]
        uv = (v * u).sum(dim=1)            # [B]
        pos_loss = torch.nn.functional.logsigmoid(uv).mean()

        neg_u = self.out_emb(neg)          # [B, K, D]
        neg_uv = torch.bmm(neg_u, v.unsqueeze(2)).squeeze(2)  # [B, K]
        neg_loss = torch.nn.functional.logsigmoid(-neg_uv).mean()
        return -(pos_loss + neg_loss)      # minimize

def train_skipgram_ns(pairs, id2idx, epochs=3, batch_size=8192, dim=64, neg_k=5, lr=0.025):
    # 노드 인덱싱
    vocab = sorted(id2idx.keys())
    vocab_size = len(vocab)

    # 빈도 기반 네거티브 샘플링 분포(0.75 승)
    counts = defaultdict(int)
    for a,b in pairs:
        counts[a] += 1; counts[b] += 1
    idx_counts = np.zeros(vocab_size, dtype=np.float64)
    for nid, cnt in counts.items():
        idx_counts[id2idx[nid]] = cnt
    prob = idx_counts ** 0.75
    prob = prob / prob.sum()
    alias_table = np.cumsum(prob)

    def sample_neg(B, K):
        # 누적분포 기반 벡터화 샘플링
        r = np.random.rand(B, K)
        idx = np.searchsorted(alias_table, r, side="right")
        return torch.from_numpy(idx.astype(np.int64))

    # 학습 데이터 텐서화(인덱스 변환)
    centers = torch.tensor([id2idx[a] for a,_ in pairs], dtype=torch.long)
    contexts= torch.tensor([id2idx[b] for _,b in pairs], dtype=torch.long)

    ds_size = len(pairs)
    model = SkipGramNS(vocab_size, dim).to(DEVICE)
    optim = torch.optim.SGD(model.parameters(), lr=lr)

    for epoch in range(1, epochs+1):
        perm = torch.randperm(ds_size)
        centers = centers[perm]; contexts = contexts[perm]
        total_loss = 0.0; steps = 0

        for i in range(0, ds_size, batch_size):
            c_batch = centers[i:i+batch_size].to(DEVICE)
            p_batch = contexts[i:i+batch_size].to(DEVICE)
            B = c_batch.size(0)
            n_batch = sample_neg(B, neg_k).to(DEVICE)

            loss = model(c_batch, p_batch, n_batch)
            optim.zero_grad(); loss.backward(); optim.step()

            total_loss += loss.item(); steps += 1

        avg = total_loss / max(1, steps)
        print(f"[Epoch {epoch}/{epochs}] loss={avg:.4f}")

    with torch.no_grad():
        emb = model.in_emb.weight.detach().cpu().numpy()
    return emb  # shape [vocab_size, dim]

def main():
    print("📥 그래프 로딩...")
    adj, cumsums, nodes = load_graph(EDGES_PATH)
    print(f"nodes={len(nodes):,}, edges(undirected)≈{sum(len(v) for v in adj.values())//2:,}")

    print("🚶 랜덤워크 생성...")
    walks = node2vec_walks(
        adj, cumsums, nodes,
        walk_length=WALK_LENGTH,
        num_walks=NUM_WALKS,
        p=P_RETURN, q=Q_INOUT
    )
    avg_len = np.mean([len(w) for w in walks]) if walks else 0
    print(f"walks={len(walks):,}, avg_len≈{avg_len:.1f}")

    print("🧩 학습 쌍 생성(Skip-gram window)...")
    pairs = generate_pairs(walks, WINDOW)
    print(f"pairs={len(pairs):,}")

    # 노드 id → 연속 index 매핑
    id2idx = {nid:i for i, nid in enumerate(sorted(nodes))}
    print("🧠 Skip-gram(NS) 학습 시작...")
    emb = train_skipgram_ns(
        pairs, id2idx,
        epochs=EPOCHS, batch_size=BATCH_SIZE,
        dim=DIM, neg_k=NEGATIVE_K, lr=LR
    )

    # CSV 저장
    idx2id = {i:nid for nid,i in id2idx.items()}
    rows = [[idx2id[i]] + list(map(float, emb[i])) for i in range(len(idx2id))]
    cols = ["content_id"] + [f"emb_{i}" for i in range(DIM)]
    out = pd.DataFrame(rows, columns=cols).sort_values("content_id")
    out.to_csv(OUT_EMB_CSV, index=False, encoding="utf-8-sig")
    print(f"✅ 임베딩 저장: {OUT_EMB_CSV} (nodes={len(out)}, dim={DIM})")

if __name__ == "__main__":
    main()


📥 그래프 로딩...
nodes=495, edges(undirected)≈31,148
🚶 랜덤워크 생성...
walks=4,950, avg_len≈40.0
🧩 학습 쌍 생성(Skip-gram window)...
pairs=1,831,500
🧠 Skip-gram(NS) 학습 시작...
[Epoch 1/3] loss=1.3863
[Epoch 2/3] loss=1.3863
[Epoch 3/3] loss=1.3863
✅ 임베딩 저장: csv 데이터\clean\item_embeddings_torch.csv (nodes=495, dim=64)


# 유저 임베딩 생성

In [59]:
# -*- coding: utf-8 -*-
# 유저 임베딩 생성 (아이템 임베딩 + 선호 도메인/장르 centroid)

import os
import numpy as np
import pandas as pd
from typing import Optional, List

BASE = r"csv 데이터\clean"
ITEM_EMB_1 = os.path.join(BASE, "item_embeddings_torch.csv")
ITEM_EMB_2 = os.path.join(BASE, "item_embeddings.csv")
CONTENTS   = os.path.join(BASE, "contents.csv")
RAW_GENRES = os.path.join(BASE, "content_raw_genres.csv")
UPREF      = os.path.join(BASE, "user_preferences.csv")
UP_CTYPES  = os.path.join(BASE, "user_preferred_content_types.csv")
UP_GENRES  = os.path.join(BASE, "user_preferred_genres.csv")
OUT_USER   = os.path.join(BASE, "user_embeddings.csv")

def read_csv_retry(path, encodings=("utf-8-sig","utf-8","cp949","euc-kr","latin1"), **kw) -> Optional[pd.DataFrame]:
    if not os.path.exists(path):
        return None
    last = None
    for enc in encodings:
        try:
            return pd.read_csv(path, encoding=enc, **kw)
        except Exception as e:
            last = e
    raise last

def load_item_embeddings() -> pd.DataFrame:
    # 명시적 선택 (DataFrame에 or 금지)
    df1 = read_csv_retry(ITEM_EMB_1)
    if df1 is not None and not df1.empty:
        df = df1
        src = os.path.basename(ITEM_EMB_1)
    else:
        df2 = read_csv_retry(ITEM_EMB_2)
        if df2 is not None and not df2.empty:
            df = df2
            src = os.path.basename(ITEM_EMB_2)
        else:
            raise RuntimeError("아이템 임베딩 CSV가 없습니다: item_embeddings_torch.csv 또는 item_embeddings.csv")

    if "content_id" not in df.columns:
        raise RuntimeError("아이템 임베딩 CSV에 content_id 컬럼이 필요합니다.")

    emb_cols = [c for c in df.columns if c.startswith("emb_")]
    if not emb_cols:
        raise RuntimeError("아이템 임베딩 CSV에 emb_로 시작하는 임베딩 컬럼이 없습니다.")

    df["content_id"] = pd.to_numeric(df["content_id"], errors="coerce").astype("Int64")
    df = df[df["content_id"].notna()].copy()
    df["content_id"] = df["content_id"].astype(int)

    for c in emb_cols:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    df.dropna(subset=emb_cols, inplace=True)

    # L2 normalize
    M = df[emb_cols].to_numpy(np.float32)
    M = M / (np.linalg.norm(M, axis=1, keepdims=True) + 1e-12)
    df.loc[:, emb_cols] = M

    print(f"✅ 아이템 임베딩 로드: {src} (items={len(df)}, dim={len(emb_cols)})")
    return df[["content_id"] + emb_cols]

# content_type → domain 매핑
CTYPE2DOMAIN = {
    "movie": "AV", "tv": "AV",
    "game": "GAME",
    "webnovel": "WEBNOVEL", "webtoon": "WEBNOVEL"
}

def main():
    emb = load_item_embeddings()
    emb_cols = [c for c in emb.columns if c.startswith("emb_")]
    M = emb[emb_cols].to_numpy(np.float32)
    ids = emb["content_id"].to_numpy(int)

    # contents.csv → domain
    contents = read_csv_retry(CONTENTS)
    if contents is None or contents.empty:
        raise RuntimeError("contents.csv 필요.")
    contents = contents[["content_id", "domain"]].copy()
    contents["content_id"] = pd.to_numeric(contents["content_id"], errors="coerce").astype("Int64")
    contents = contents[contents["content_id"].notna()].copy()
    contents["content_id"] = contents["content_id"].astype(int)
    emb_meta = emb.merge(contents, on="content_id", how="left")

    # domain centroids
    domain_vecs = {}
    dom_counts = {}
    for dom, grp in emb_meta.dropna(subset=["domain"]).groupby("domain"):
        mat = grp[emb_cols].to_numpy(np.float32)
        if len(mat) == 0: 
            continue
        v = mat.mean(axis=0)
        v = v / (np.linalg.norm(v) + 1e-12)
        domain_vecs[str(dom)] = v
        dom_counts[str(dom)] = len(mat)
    print(f"✅ 도메인 센트로이드: { {k: dom_counts[k] for k in sorted(dom_counts)} }")

    # genre centroids (optional)
    genre_vecs = {}
    if os.path.exists(RAW_GENRES):
        rawg = read_csv_retry(RAW_GENRES)
        if rawg is not None and not rawg.empty and {"content_id","raw_genre"}.issubset(rawg.columns):
            rawg = rawg[["content_id", "raw_genre"]].dropna().copy()
            rawg["content_id"] = pd.to_numeric(rawg["content_id"], errors="coerce").astype("Int64")
            rawg = rawg[rawg["content_id"].notna()].copy()
            rawg["content_id"] = rawg["content_id"].astype(int)
            rawg["g"] = rawg["raw_genre"].astype(str).str.strip().str.lower()

            idx_map = {cid: i for i, cid in enumerate(ids)}
            for g, grp in rawg.groupby("g"):
                idxs = [idx_map[c] for c in grp["content_id"].tolist() if c in idx_map]
                if not idxs:
                    continue
                v = M[idxs].mean(axis=0)
                v = v / (np.linalg.norm(v) + 1e-12)
                genre_vecs[g] = v
            print(f"✅ 장르 센트로이드 수: {len(genre_vecs)}")
        else:
            print("ℹ️ content_raw_genres.csv 비어있음 또는 컬럼 불충분 → 장르 센트로이드 생략")
    else:
        print("ℹ️ content_raw_genres.csv 없음 → 장르 센트로이드 생략")

    # user prefs
    upref = read_csv_retry(UPREF)
    uctype = read_csv_retry(UP_CTYPES)
    ugen = read_csv_retry(UP_GENRES)
    if upref is None or upref.empty:
        raise RuntimeError("user_preferences.csv 필요.")

    ctype_map = {}
    if uctype is not None and not uctype.empty and {"preference_id", "content_type"}.issubset(uctype.columns):
        for pid, grp in uctype.groupby("preference_id"):
            ctype_map[int(pid)] = [str(x).strip() for x in grp["content_type"].tolist()]

    genre_map = {}
    if ugen is not None and not ugen.empty and {"preference_id", "genre"}.issubset(ugen.columns):
        for pid, grp in ugen.groupby("preference_id"):
            genre_map[int(pid)] = [str(x).strip().lower() for x in grp["genre"].tolist()]

    def ctypes_to_domains(cts: List[str]) -> List[str]:
        outs = set()
        for ct in cts:
            d = CTYPE2DOMAIN.get(str(ct).lower())
            if d:
                outs.add(d)
        return sorted(outs)

    # 가중치
    ALPHA_DOM, ALPHA_GEN = 0.6, 0.4

    rows = []
    skipped = 0
    for _, r in upref.iterrows():
        if pd.isna(r.get("id")):
            skipped += 1
            continue
        uid = int(r["id"])
        uname = r.get("username", "")

        # domains
        doms = ctypes_to_domains(ctype_map.get(uid, []))
        dom_vecs = [domain_vecs[d] for d in doms if d in domain_vecs]
        v_dom = None
        if dom_vecs:
            v_dom = np.mean(np.stack(dom_vecs, axis=0), axis=0)
            v_dom = v_dom / (np.linalg.norm(v_dom) + 1e-12)

        # genres
        gnames = [g for g in (genre_map.get(uid, [])) if g in genre_vecs]
        g_vecs = [genre_vecs[g] for g in gnames]
        v_gen = None
        if g_vecs:
            v_gen = np.mean(np.stack(g_vecs, axis=0), axis=0)
            v_gen = v_gen / (np.linalg.norm(v_gen) + 1e-12)

        # user vector
        if v_dom is None and v_gen is None:
            # 폴백: 전체 아이템 평균
            v_user = M.mean(axis=0)
        elif v_dom is None:
            v_user = v_gen
        elif v_gen is None:
            v_user = v_dom
        else:
            v_user = ALPHA_DOM * v_dom + ALPHA_GEN * v_gen
        v_user = v_user / (np.linalg.norm(v_user) + 1e-12)

        rows.append([uid, uname] + list(map(float, v_user)))

    if not rows:
        raise RuntimeError("유저 임베딩 생성 결과가 비었습니다. 입력 파일/선호 정보 확인 필요.")

    dim = M.shape[1]
    out = pd.DataFrame(rows, columns=["user_id", "username"] + [f"emb_{i}" for i in range(dim)])
    out.sort_values("user_id").to_csv(OUT_USER, index=False, encoding="utf-8-sig")
    print(f"✅ user_embeddings.csv 저장: {OUT_USER} (users={len(out)}, skipped_rows={skipped})")

if __name__ == "__main__":
    main()


✅ 아이템 임베딩 로드: item_embeddings_torch.csv (items=495, dim=64)
✅ 도메인 센트로이드: {'AV': 200, 'GAME': 100, 'WEBNOVEL': 195}
ℹ️ content_raw_genres.csv 비어있음 또는 컬럼 불충분 → 장르 센트로이드 생략
✅ user_embeddings.csv 저장: csv 데이터\clean\user_embeddings.csv (users=10, skipped_rows=0)


# 유저 임베딩 생성(장르만)

In [8]:
# -*- coding: utf-8 -*-
# 유저 임베딩 생성 (유저 선호 장르 centroid만 사용)

import os
import numpy as np
import pandas as pd
from typing import Optional, List

BASE = r"csv 데이터\clean"
ITEM_EMB_1 = os.path.join(BASE, "item_embeddings_torch.csv")
ITEM_EMB_2 = os.path.join(BASE, "item_embeddings.csv")
CONTENTS   = os.path.join(BASE, "contents.csv")
RAW_GENRES = os.path.join(BASE, "content_raw_genres.csv")
UP_GENRES  = os.path.join(BASE, "user_preferred_genres.csv")
OUT_USER   = os.path.join(BASE, "user_embeddings.csv")


def read_csv_retry(path, encodings=("utf-8-sig","utf-8","cp949","euc-kr","latin1"), **kw) -> Optional[pd.DataFrame]:
    if not os.path.exists(path):
        return None
    last = None
    for enc in encodings:
        try:
            return pd.read_csv(path, encoding=enc, **kw)
        except Exception as e:
            last = e
    raise last


def load_item_embeddings() -> pd.DataFrame:
    # 명시적 선택 (DataFrame에 or 금지)
    df1 = read_csv_retry(ITEM_EMB_1)
    if df1 is not None and not df1.empty:
        df = df1
        src = os.path.basename(ITEM_EMB_1)
    else:
        df2 = read_csv_retry(ITEM_EMB_2)
        if df2 is not None and not df2.empty:
            df = df2
            src = os.path.basename(ITEM_EMB_2)
        else:
            raise RuntimeError("아이템 임베딩 CSV가 없습니다: item_embeddings_torch.csv 또는 item_embeddings.csv")

    if "content_id" not in df.columns:
        raise RuntimeError("아이템 임베딩 CSV에 content_id 컬럼이 필요합니다.")

    emb_cols = [c for c in df.columns if c.startswith("emb_")]
    if not emb_cols:
        raise RuntimeError("아이템 임베딩 CSV에 emb_로 시작하는 임베딩 컬럼이 없습니다.")

    df["content_id"] = pd.to_numeric(df["content_id"], errors="coerce").astype("Int64")
    df = df[df["content_id"].notna()].copy()
    df["content_id"] = df["content_id"].astype(int)

    for c in emb_cols:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    df.dropna(subset=emb_cols, inplace=True)

    # L2 normalize
    M = df[emb_cols].to_numpy(np.float32)
    M = M / (np.linalg.norm(M, axis=1, keepdims=True) + 1e-12)
    df.loc[:, emb_cols] = M

    print(f"✅ 아이템 임베딩 로드: {src} (items={len(df)}, dim={len(emb_cols)})")
    return df[["content_id"] + emb_cols]


def main():
    # 1) 아이템 임베딩 + contents 로드
    emb = load_item_embeddings()
    emb_cols = [c for c in emb.columns if c.startswith("emb_")]
    M = emb[emb_cols].to_numpy(np.float32)
    ids = emb["content_id"].to_numpy(int)

    contents = read_csv_retry(CONTENTS)
    if contents is None or contents.empty:
        raise RuntimeError("contents.csv 필요.")
    contents = contents[["content_id", "domain"]].copy()
    contents["content_id"] = pd.to_numeric(contents["content_id"], errors="coerce").astype("Int64")
    contents = contents[contents["content_id"].notna()].copy()
    contents["content_id"] = contents["content_id"].astype(int)
    emb_meta = emb.merge(contents, on="content_id", how="left")

    # 2) (옵션) domain 정보는 더 이상 안 쓰지만, 필요하면 참고용으로 센트로이드 계산
    dom_counts = {}
    for dom, grp in emb_meta.dropna(subset=["domain"]).groupby("domain"):
        dom_counts[str(dom)] = len(grp)
    if dom_counts:
        print(f"ℹ️ 도메인별 아이템 수 (참고용): { {k: dom_counts[k] for k in sorted(dom_counts)} }")

    # 3) 장르 센트로이드 계산 (content_raw_genres.csv + item 임베딩)
    genre_vecs = {}
    if os.path.exists(RAW_GENRES):
        rawg = read_csv_retry(RAW_GENRES)
        if rawg is not None and not rawg.empty and {"content_id","raw_genre"}.issubset(rawg.columns):
            rawg = rawg[["content_id", "raw_genre"]].dropna().copy()
            rawg["content_id"] = pd.to_numeric(rawg["content_id"], errors="coerce").astype("Int64")
            rawg = rawg[rawg["content_id"].notna()].copy()
            rawg["content_id"] = rawg["content_id"].astype(int)
            rawg["g"] = rawg["raw_genre"].astype(str).str.strip().str.lower()

            idx_map = {cid: i for i, cid in enumerate(ids)}
            for g, grp in rawg.groupby("g"):
                idxs = [idx_map[c] for c in grp["content_id"].tolist() if c in idx_map]
                if not idxs:
                    continue
                v = M[idxs].mean(axis=0)
                v = v / (np.linalg.norm(v) + 1e-12)
                genre_vecs[g] = v
            print(f"✅ 장르 센트로이드 수: {len(genre_vecs)}")
        else:
            raise RuntimeError("content_raw_genres.csv 비어있거나 컬럼 부족(raw_genre, content_id) → 장르 센트로이드 계산 불가")
    else:
        raise RuntimeError("content_raw_genres.csv 없음 → 장르 센트로이드 계산 불가")

    # 4) user_preferred_genres.csv 로부터 유저별 선호 장르 가져오기
    ugen = read_csv_retry(UP_GENRES)
    if ugen is None or ugen.empty:
        raise RuntimeError("user_preferred_genres.csv 필요.")

    # 어떤 컬럼이 유저 식별자인지 추론 (user_id / id / preference_id 중 하나)
    col_user_id = None
    for cand in ["user_id", "id", "preference_id"]:
        if cand in ugen.columns:
            col_user_id = cand
            break
    if col_user_id is None:
        raise RuntimeError("user_preferred_genres.csv 에 user_id / id / preference_id 컬럼이 없습니다. 유저 식별 컬럼 하나 필요.")

    if "genre" not in ugen.columns:
        raise RuntimeError("user_preferred_genres.csv 에 genre 컬럼이 필요합니다.")

    has_username = "username" in ugen.columns

    keep_cols = [col_user_id, "genre"]
    if has_username:
        keep_cols.append("username")

    ugen = ugen[keep_cols].dropna(subset=[col_user_id, "genre"]).copy()
    ugen[col_user_id] = pd.to_numeric(ugen[col_user_id], errors="coerce").astype("Int64")
    ugen = ugen[ugen[col_user_id].notna()].copy()
    ugen[col_user_id] = ugen[col_user_id].astype(int)
    ugen["g"] = ugen["genre"].astype(str).str.strip().str.lower()

    # 5) 유저별로 선호 장르 → 장르 벡터 평균 → 유저 임베딩
    rows = []
    skipped = 0
    for uid, grp in ugen.groupby(col_user_id):
        # 이 유저가 선호한다고 표시한 장르들 중, 실제 장르 센트로이드가 있는 것만 사용
        genres = sorted(set([g for g in grp["g"].tolist() if g in genre_vecs]))
        if not genres:
            # 장르 문자열은 있지만 content_raw_genres 기반 센트로이드가 없으면 스킵
            skipped += 1
            continue

        g_vecs = [genre_vecs[g] for g in genres]
        v_user = np.mean(np.stack(g_vecs, axis=0), axis=0)
        v_user = v_user / (np.linalg.norm(v_user) + 1e-12)

        uname = ""
        if has_username:
            # 같은 유저에 username 여러 개 있어도 첫 번째 것 사용
            uname = str(grp["username"].iloc[0])

        rows.append([int(uid), uname] + list(map(float, v_user)))

    if not rows:
        raise RuntimeError("유저 임베딩 생성 결과가 비었습니다. user_preferred_genres.csv / 장르 센트로이드 내용을 확인하세요.")

    dim = len(emb_cols)
    out = pd.DataFrame(rows, columns=["user_id", "username"] + [f"emb_{i}" for i in range(dim)])
    out.sort_values("user_id").to_csv(OUT_USER, index=False, encoding="utf-8-sig")
    print(f"✅ user_embeddings.csv 저장: {OUT_USER} (users={len(out)}, skipped_users_without_valid_genres={skipped})")


if __name__ == "__main__":
    main()


✅ 아이템 임베딩 로드: item_embeddings_torch.csv (items=495, dim=64)
ℹ️ 도메인별 아이템 수 (참고용): {'AV': 200, 'GAME': 100, 'WEBNOVEL': 195}


RuntimeError: content_raw_genres.csv 비어있거나 컬럼 부족(raw_genre, content_id) → 장르 센트로이드 계산 불가

In [9]:
# -*- coding: utf-8 -*-
# 유저 임베딩 생성 (유저 선호 장르 centroid만 사용)

import os
import numpy as np
import pandas as pd
from typing import Optional, List

BASE = r"csv 데이터\clean"
ITEM_EMB_1 = os.path.join(BASE, "item_embeddings_torch.csv")
ITEM_EMB_2 = os.path.join(BASE, "item_embeddings.csv")
CONTENTS   = os.path.join(BASE, "contents.csv")
RAW_GENRES = os.path.join(BASE, "content_raw_genres.csv")  # 지금: content_id, source, raw_genre_1~3
UP_GENRES  = os.path.join(BASE, "user_preferred_genres.csv")
OUT_USER   = os.path.join(BASE, "user_embeddings.csv")


def read_csv_retry(path, encodings=("utf-8-sig","utf-8","cp949","euc-kr","latin1"), **kw) -> Optional[pd.DataFrame]:
    if not os.path.exists(path):
        return None
    last = None
    for enc in encodings:
        try:
            return pd.read_csv(path, encoding=enc, **kw)
        except Exception as e:
            last = e
    raise last


def load_item_embeddings() -> pd.DataFrame:
    # 명시적 선택 (DataFrame에 or 금지)
    df1 = read_csv_retry(ITEM_EMB_1)
    if df1 is not None and not df1.empty:
        df = df1
        src = os.path.basename(ITEM_EMB_1)
    else:
        df2 = read_csv_retry(ITEM_EMB_2)
        if df2 is not None and not df2.empty:
            df = df2
            src = os.path.basename(ITEM_EMB_2)
        else:
            raise RuntimeError("아이템 임베딩 CSV가 없습니다: item_embeddings_torch.csv 또는 item_embeddings.csv")

    if "content_id" not in df.columns:
        raise RuntimeError("아이템 임베딩 CSV에 content_id 컬럼이 필요합니다.")

    emb_cols = [c for c in df.columns if c.startswith("emb_")]
    if not emb_cols:
        raise RuntimeError("아이템 임베딩 CSV에 emb_로 시작하는 임베딩 컬럼이 없습니다.")

    df["content_id"] = pd.to_numeric(df["content_id"], errors="coerce").astype("Int64")
    df = df[df["content_id"].notna()].copy()
    df["content_id"] = df["content_id"].astype(int)

    for c in emb_cols:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    df.dropna(subset=emb_cols, inplace=True)

    # L2 normalize
    M = df[emb_cols].to_numpy(np.float32)
    M = M / (np.linalg.norm(M, axis=1, keepdims=True) + 1e-12)
    df.loc[:, emb_cols] = M

    print(f"✅ 아이템 임베딩 로드: {src} (items={len(df)}, dim={len(emb_cols)})")
    return df[["content_id"] + emb_cols]


def load_content_genres_tall() -> pd.DataFrame:
    """
    content_raw_genres.csv 가
      - 세로형: content_id, source, raw_genre
      - 또는 가로형: content_id, source, raw_genre_1~3
    둘 중 어떤 포맷이든 받아서
    항상 (content_id, raw_genre) 세로형으로 반환.
    """
    g = read_csv_retry(RAW_GENRES)
    if g is None or g.empty:
        raise RuntimeError("content_raw_genres.csv 가 비어있거나 없습니다.")

    g = g.copy()
    g.columns = [c.strip() for c in g.columns]

    if "content_id" not in g.columns:
        raise RuntimeError("content_raw_genres.csv 에 content_id 컬럼이 없습니다.")

    # 이미 raw_genre 단일 컬럼이 있으면 그대로 사용
    if "raw_genre" in g.columns:
        g["content_id"] = pd.to_numeric(g["content_id"], errors="coerce").astype("Int64")
        g = g[g["content_id"].notna()].copy()
        g["content_id"] = g["content_id"].astype(int)
        g["raw_genre"]  = g["raw_genre"].astype(str).str.strip()
        g = g[g["raw_genre"] != ""]
        return g[["content_id", "raw_genre"]].drop_duplicates().reset_index(drop=True)

    # 가로형: raw_genre_1~3 을 세로형으로 펴기
    genre_cols = [c for c in ["raw_genre_1", "raw_genre_2", "raw_genre_3"] if c in g.columns]
    if not genre_cols:
        raise RuntimeError("content_raw_genres.csv 에 raw_genre 또는 raw_genre_1/2/3 컬럼이 필요합니다.")

    g["content_id"] = pd.to_numeric(g["content_id"], errors="coerce").astype("Int64")
    g = g[g["content_id"].notna()].copy()
    g["content_id"] = g["content_id"].astype(int)

    tall = g.melt(
        id_vars=["content_id"],
        value_vars=genre_cols,
        value_name="raw_genre"
    )
    tall["raw_genre"] = tall["raw_genre"].astype(str).str.strip()
    tall = tall[tall["raw_genre"].notna() & (tall["raw_genre"] != "")]
    tall = tall[["content_id", "raw_genre"]].drop_duplicates()

    print(f"✅ content_raw_genres wide → tall 변환: rows={len(tall)}")
    return tall.reset_index(drop=True)


def main():
    # 1) 아이템 임베딩 + contents 로드
    emb = load_item_embeddings()
    emb_cols = [c for c in emb.columns if c.startswith("emb_")]
    M = emb[emb_cols].to_numpy(np.float32)
    ids = emb["content_id"].to_numpy(int)

    contents = read_csv_retry(CONTENTS)
    if contents is None or contents.empty:
        raise RuntimeError("contents.csv 필요.")
    contents = contents[["content_id", "domain"]].copy()
    contents["content_id"] = pd.to_numeric(contents["content_id"], errors="coerce").astype("Int64")
    contents = contents[contents["content_id"].notna()].copy()
    contents["content_id"] = contents["content_id"].astype(int)
    emb_meta = emb.merge(contents, on="content_id", how="left")

    # (옵션) 도메인 정보 출력만
    dom_counts = {}
    for dom, grp in emb_meta.dropna(subset=["domain"]).groupby("domain"):
        dom_counts[str(dom)] = len(grp)
    if dom_counts:
        print(f"ℹ️ 도메인별 아이템 수 (참고용): { {k: dom_counts[k] for k in sorted(dom_counts)} }")

    # 3) 장르 센트로이드 계산 (content_raw_genres.csv + item 임베딩)
    if not os.path.exists(RAW_GENRES):
        raise RuntimeError("content_raw_genres.csv 없음 → 장르 센트로이드 계산 불가")

    rawg_tall = load_content_genres_tall()  # 항상 content_id, raw_genre 세로형으로 반환

    if rawg_tall is None or rawg_tall.empty:
        raise RuntimeError("content_raw_genres.csv 에 유효한 장르가 없습니다. → 장르 센트로이드 계산 불가")

    rawg = rawg_tall.copy()
    rawg["g"] = rawg["raw_genre"].astype(str).str.strip().str.lower()

    idx_map = {cid: i for i, cid in enumerate(ids)}
    genre_vecs = {}
    for g, grp in rawg.groupby("g"):
        idxs = [idx_map[c] for c in grp["content_id"].tolist() if c in idx_map]
        if not idxs:
            continue
        v = M[idxs].mean(axis=0)
        v = v / (np.linalg.norm(v) + 1e-12)
        genre_vecs[g] = v

    if not genre_vecs:
        raise RuntimeError("장르별로 매칭되는 아이템 임베딩이 없습니다. genre_vecs 비어있음.")

    print(f"✅ 장르 센트로이드 수: {len(genre_vecs)}")

    # 4) user_preferred_genres.csv 로부터 유저별 선호 장르 가져오기
    ugen = read_csv_retry(UP_GENRES)
    if ugen is None or ugen.empty:
        raise RuntimeError("user_preferred_genres.csv 필요.")

    # 어떤 컬럼이 유저 식별자인지 추론 (user_id / id / preference_id 중 하나)
    col_user_id = None
    for cand in ["user_id", "id", "preference_id"]:
        if cand in ugen.columns:
            col_user_id = cand
            break
    if col_user_id is None:
        raise RuntimeError("user_preferred_genres.csv 에 user_id / id / preference_id 컬럼이 없습니다. 유저 식별 컬럼 하나 필요.")

    if "genre" not in ugen.columns:
        raise RuntimeError("user_preferred_genres.csv 에 genre 컬럼이 필요합니다.")

    has_username = "username" in ugen.columns

    keep_cols = [col_user_id, "genre"]
    if has_username:
        keep_cols.append("username")

    ugen = ugen[keep_cols].dropna(subset=[col_user_id, "genre"]).copy()
    ugen[col_user_id] = pd.to_numeric(ugen[col_user_id], errors="coerce").astype("Int64")
    ugen = ugen[ugen[col_user_id].notna()].copy()
    ugen[col_user_id] = ugen[col_user_id].astype(int)
    ugen["g"] = ugen["genre"].astype(str).str.strip().str.lower()

    # 5) 유저별로 선호 장르 → 장르 벡터 평균 → 유저 임베딩
    rows = []
    skipped = 0
    for uid, grp in ugen.groupby(col_user_id):
        genres = sorted(set([g for g in grp["g"].tolist() if g in genre_vecs]))
        if not genres:
            skipped += 1
            continue

        g_vecs = [genre_vecs[g] for g in genres]
        v_user = np.mean(np.stack(g_vecs, axis=0), axis=0)
        v_user = v_user / (np.linalg.norm(v_user) + 1e-12)

        uname = ""
        if has_username:
            uname = str(grp["username"].iloc[0])

        rows.append([int(uid), uname] + list(map(float, v_user)))

    if not rows:
        raise RuntimeError("유저 임베딩 생성 결과가 비었습니다. user_preferred_genres.csv / 장르 센트로이드 내용을 확인하세요.")

    dim = len(emb_cols)
    out = pd.DataFrame(rows, columns=["user_id", "username"] + [f"emb_{i}" for i in range(dim)])
    out.sort_values("user_id").to_csv(OUT_USER, index=False, encoding="utf-8-sig")
    print(f"✅ user_embeddings.csv 저장: {OUT_USER} (users={len(out)}, skipped_users_without_valid_genres={skipped})")


if __name__ == "__main__":
    main()


✅ 아이템 임베딩 로드: item_embeddings_torch.csv (items=495, dim=64)
ℹ️ 도메인별 아이템 수 (참고용): {'AV': 200, 'GAME': 100, 'WEBNOVEL': 195}
✅ content_raw_genres wide → tall 변환: rows=801
✅ 장르 센트로이드 수: 45
✅ user_embeddings.csv 저장: csv 데이터\clean\user_embeddings.csv (users=10, skipped_users_without_valid_genres=0)


# 유저 아이템 임베딩 top-k추천

In [10]:
# -*- coding: utf-8 -*-

#유저/아이템 임베딩만으로 Top-K 추천 (contents.csv에서 이름/도메인까지 붙이기)
#입력:
#  clean/user_embeddings.csv                  (user_id, username, emb_0..)
#  clean/item_embeddings_torch.csv 또는 item_embeddings.csv (content_id, emb_0..)
#  clean/contents.csv                         (content_id, master_title, domain)
#  clean/user_preferences.csv                 (선택) id, username  ← 추가로 사용
#출력:
#  clean/recommendations_topK.csv             (user_id, username, rank, content_id, master_title, domain, score)

import os
import re
import numpy as np
import pandas as pd
from typing import Optional

BASE = r"csv 데이터\clean"
USER_EMB   = os.path.join(BASE, "user_embeddings.csv")
ITEM_EMB_1 = os.path.join(BASE, "item_embeddings_torch.csv")
ITEM_EMB_2 = os.path.join(BASE, "item_embeddings.csv")
CONTENTS   = os.path.join(BASE, "contents.csv")
UPREF      = os.path.join(BASE, "user_preferences.csv")   # ★ 추가
OUT_RECS   = os.path.join(BASE, "recommendations_topK.csv")

TOPK = 5
EXCLUDE_DUP_CONTENTS = True   # content_id 중복행이 있으면 첫 번째만 사용
ASSUME_NORMALIZED = True      # 임베딩이 이미 L2 정규화되어 있다면 True, 아니면 False로

def read_csv_retry(path, encodings=("utf-8-sig","utf-8","cp949","euc-kr","latin1"), **kw) -> Optional[pd.DataFrame]:
    if not os.path.exists(path):
        return None
    last=None
    for enc in encodings:
        try:
            return pd.read_csv(path, encoding=enc, **kw)
        except Exception as e:
            last=e
    raise last

def l2norm(X: np.ndarray) -> np.ndarray:
    return X / (np.linalg.norm(X, axis=1, keepdims=True) + 1e-12)

def _coerce_content_id_column(df: pd.DataFrame) -> pd.Series:
    """여러 이름/형태로 저장된 content_id를 최대한 복구."""
    df = df.copy()
    orig_cols = list(df.columns)
    df.columns = [c.strip() for c in df.columns]
    name_map = {c.lower(): c for c in df.columns}
    for key in ["content_id", "contentid", "content id", "id", "unnamed: 0"]:
        if key in name_map:
            cand = name_map[key]
            s = df[cand]
            if s.dtype == object:
                s = s.astype(str).str.strip()
            s = pd.to_numeric(s, errors="coerce").astype("Int64")
            if s.isna().all():
                continue
            return s
    raise RuntimeError(f"content_id 열을 찾지 못했습니다. 원본 컬럼: {orig_cols}")

def load_user_embeddings() -> pd.DataFrame:
    u = read_csv_retry(USER_EMB)
    if u is None or u.empty:
        raise RuntimeError("user_embeddings.csv 필요.")
    u = u.copy()
    u.columns = [c.strip() for c in u.columns]

    # user_id 정리
    if "user_id" not in u.columns:
        if "id" in u.columns:
            u = u.rename(columns={"id": "user_id"})
        else:
            raise RuntimeError("user_embeddings.csv에 user_id 컬럼 필요.")
    u["user_id"] = pd.to_numeric(u["user_id"], errors="coerce").astype("Int64")
    u = u[u["user_id"].notna()].copy()
    u["user_id"] = u["user_id"].astype(int)

    # 임베딩 컬럼
    emb_cols = [c for c in u.columns if c.lower().startswith("emb_")]
    if not emb_cols:
        raise RuntimeError("user_embeddings.csv에 emb_로 시작하는 임베딩 컬럼이 없습니다.")
    for c in emb_cols:
        u[c] = pd.to_numeric(u[c], errors="coerce")
    u.dropna(subset=emb_cols, inplace=True)
    if not ASSUME_NORMALIZED:
        u.loc[:, emb_cols] = l2norm(u[emb_cols].to_numpy(np.float32))

    # username 컬럼 찾기 (대소문자 무시)
    lower_cols = [x.lower() for x in u.columns]
    if "username" in lower_cols:
        uname_col = u.columns[lower_cols.index("username")]
        cols = ["user_id", uname_col] + emb_cols
        out = u[cols].copy()
        if uname_col != "username":
            out = out.rename(columns={uname_col: "username"})
    else:
        cols = ["user_id"] + emb_cols
        out = u[cols].copy()
        out["username"] = ""

    # 🔹 1단계: user_preferences.csv에서 username 보강
    pref = read_csv_retry(UPREF)
    if pref is not None and not pref.empty:
        pref = pref.copy()
        pref.columns = [c.strip() for c in pref.columns]
        lower = {c.lower(): c for c in pref.columns}

        id_col    = lower.get("id") or lower.get("user_id")
        uname_col = lower.get("username") or lower.get("user_name")

        if id_col is not None and uname_col is not None:
            pref = pref[[id_col, uname_col]].dropna(subset=[id_col])
            pref[id_col] = pd.to_numeric(pref[id_col], errors="coerce").astype("Int64")
            pref = pref[pref[id_col].notna()].copy()
            pref[id_col] = pref[id_col].astype(int)
            pref = pref.rename(columns={id_col: "user_id", uname_col: "username_pref"})

            out = out.merge(pref, on="user_id", how="left")
            # user_embeddings의 username이 비어있으면 preferences의 username_pref로 채우기
            out["username"] = out["username"].astype(str)
            out["username_pref"] = out["username_pref"].astype(str)

            # embeddings 쪽 username이 없거나 "nan"/"None" 같은 값이면 pref 것으로 덮어쓰기
            mask_missing = out["username"].str.strip().eq("") | \
                           out["username"].str.strip().str.lower().isin(["nan", "none"])
            out.loc[mask_missing, "username"] = out.loc[mask_missing, "username_pref"]
            out = out.drop(columns=["username_pref"])

    # 🔹 2단계: 최종적으로 남은 이상값 정리
    out["username"] = out["username"].replace({np.nan: ""})
    out["username"] = out["username"].astype(str)
    bad = out["username"].str.strip().str.lower().isin(["nan", "none"])
    out.loc[bad, "username"] = ""

    return out[["user_id", "username"] + emb_cols]

def load_item_embeddings() -> pd.DataFrame:
    i1 = read_csv_retry(ITEM_EMB_1)
    if i1 is not None and not i1.empty:
        df = i1; src = os.path.basename(ITEM_EMB_1)
    else:
        i2 = read_csv_retry(ITEM_EMB_2)
        if i2 is None or i2.empty:
            raise RuntimeError("item_embeddings CSV 필요: item_embeddings_torch.csv 또는 item_embeddings.csv")
        df = i2; src = os.path.basename(ITEM_EMB_2)

    df = df.copy()
    df.columns = [c.strip() for c in df.columns]
    df["content_id"] = _coerce_content_id_column(df)

    emb_cols = [c for c in df.columns if c.lower().startswith("emb_")]
    if not emb_cols:
        alt = [c for c in df.columns if re.search(r"\d+$", c)]
        raise RuntimeError(f"임베딩 컬럼(emb_*) 없음. 확인 필요. 후보: {alt}")

    for c in emb_cols:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    df = df.dropna(subset=emb_cols + ["content_id"]).copy()
    df["content_id"] = df["content_id"].astype(int)

    if EXCLUDE_DUP_CONTENTS:
        df = df.drop_duplicates(subset=["content_id"], keep="first").reset_index(drop=True)

    if not ASSUME_NORMALIZED:
        df.loc[:, emb_cols] = l2norm(df[emb_cols].to_numpy(np.float32))

    print(f"✅ 아이템 임베딩 로드: {src} (items={len(df)}, dim={len(emb_cols)})")
    return df[["content_id"] + emb_cols]

def load_contents_meta() -> Optional[pd.DataFrame]:
    """
    contents.csv에서 content_id, master_title, domain만 뽑아서 리턴.
    content_id는 위의 _coerce_content_id_column 로 통일.
    """
    c = read_csv_retry(CONTENTS)
    if c is None or c.empty:
        print("⚠️ contents.csv를 찾지 못해 이름/도메인 매핑 없이 저장합니다.")
        return None

    c = c.copy()
    c.columns = [col.strip() for col in c.columns]

    # content_id 강제 추출
    c["content_id"] = _coerce_content_id_column(c)
    c = c[c["content_id"].notna()].copy()
    c["content_id"] = c["content_id"].astype(int)

    # master_title / domain 컬럼 이름(대소문자 무시) 찾기
    lower_map = {col.lower(): col for col in c.columns}
    title_col = lower_map.get("master_title")
    domain_col = lower_map.get("domain")

    if title_col is None:
        raise RuntimeError("contents.csv에서 master_title 컬럼을 찾지 못했습니다.")
    if domain_col is None:
        raise RuntimeError("contents.csv에서 domain 컬럼을 찾지 못했습니다.")

    meta = c[["content_id", title_col, domain_col]].copy()
    # 컬럼 이름 통일
    if title_col != "master_title":
        meta = meta.rename(columns={title_col: "master_title"})
    if domain_col != "domain":
        meta = meta.rename(columns={domain_col: "domain"})

    meta = meta.drop_duplicates(subset=["content_id"], keep="first").reset_index(drop=True)
    print(f"✅ contents 메타 로드: {len(meta)} rows (content_id, master_title, domain)")
    return meta

def main():
    u = load_user_embeddings()
    i = load_item_embeddings()
    meta = load_contents_meta()  # 이름/도메인 매핑

    ucols = [c for c in u.columns if c.lower().startswith("emb_")]
    icols = [c for c in i.columns if c.lower().startswith("emb_")]

    U = u[ucols].to_numpy(np.float32)
    I = i[icols].to_numpy(np.float32)

    # 혹시 ASSUME_NORMALIZED=True인데 실제로 정규화 안되어 있으면, 코사인 = 내적이 아닐 수 있으니 보정
    U = l2norm(U)
    I = l2norm(I)

    item_ids = i["content_id"].to_numpy(int)
    user_ids = u["user_id"].to_numpy(int)
    usernames = u["username"].astype(str).to_numpy() if "username" in u.columns else np.array([""]*len(u))

    rec_rows = []
    for r in range(U.shape[0]):
        sims = (I @ U[r:r+1].T).reshape(-1)   # 코사인(정규화 가정)
        k = min(TOPK, len(sims))
        if k == 0:
            continue
        part = np.argpartition(-sims, k-1)[:k]
        order = part[np.argsort(-sims[part])]
        top_ids = item_ids[order]
        top_scs = sims[order]
        uid = int(user_ids[r]); uname = usernames[r] if r < len(usernames) else ""
        for rank, (cid, sc) in enumerate(zip(top_ids, top_scs), start=1):
            rec_rows.append([uid, uname, rank, int(cid), float(sc)])

    rec = pd.DataFrame(rec_rows, columns=["user_id","username","rank","content_id","score"])

    # contents 메타와 merge 해서 master_title, domain 추가
    if meta is not None:
        rec = rec.merge(meta, on="content_id", how="left")
        cols = ["user_id", "username", "rank", "content_id", "master_title", "domain", "score"]
        cols = [c for c in cols if c in rec.columns]
        rec = rec[cols]

    rec.sort_values(["user_id","rank"]).to_csv(OUT_RECS, index=False, encoding="utf-8-sig")
    print(f"✅ recommendations_topK.csv 저장: {OUT_RECS} (rows={len(rec)})")

if __name__ == "__main__":
    main()


✅ 아이템 임베딩 로드: item_embeddings_torch.csv (items=495, dim=64)
✅ contents 메타 로드: 601 rows (content_id, master_title, domain)
✅ recommendations_topK.csv 저장: csv 데이터\clean\recommendations_topK.csv (rows=50)


# 아이템 추천(장르임베딩 필터)

In [67]:
# -*- coding: utf-8 -*-

# 장르 기반 유저-아이템 Top-K 추천 (user_embeddings.csv 사용 X)
# ▶ 장르가 없는 콘텐츠는 추천 후보에서 제외
#
# 입력:
#   clean/item_embeddings_torch.csv 또는 item_embeddings.csv (content_id, emb_0..)
#   clean/contents.csv                        (선택) content_id + 제목/도메인 컬럼
#   clean/content_raw_genres.csv              (필수) content_id, raw_genre
#   clean/user_preferred_genres.csv           (유저별 선호 장르)
#   clean/user_preferences.csv                (선택) id, username 등 (있으면 username 매핑)
#
# 출력:
#   clean/recommendations_topK_genre.csv
#   (user_id, username, rank, content_id, content_name, domain, genres_str, score)

import os
import re
import numpy as np
import pandas as pd
from typing import Optional, List, Dict, Tuple

BASE        = r"csv 데이터\clean"
ITEM_EMB_1  = os.path.join(BASE, "item_embeddings_torch.csv")
ITEM_EMB_2  = os.path.join(BASE, "item_embeddings.csv")
CONTENTS    = os.path.join(BASE, "contents.csv")             # 이름/도메인 메타
RAW_GENRES  = os.path.join(BASE, "content_raw_genres.csv")   # 장르용 메타
UPREF       = os.path.join(BASE, "user_preferences.csv")     # (선택) user_id, username
UP_GENRES   = os.path.join(BASE, "user_preferred_genres.csv")# 유저 선호 장르
OUT_RECS    = os.path.join(BASE, "recommendations_topK_genre.csv")

TOPK = 5
EXCLUDE_DUP_CONTENTS = True   # content_id 중복행이 있으면 첫 번째만 사용
ASSUME_NORMALIZED = True      # 임베딩이 이미 L2 정규화되어 있다면 True, 아니면 False로

def read_csv_retry(path, encodings=("utf-8-sig","utf-8","cp949","euc-kr","latin1"), **kw) -> Optional[pd.DataFrame]:
    if not os.path.exists(path):
        return None
    last = None
    for enc in encodings:
        try:
            return pd.read_csv(path, encoding=enc, **kw)
        except Exception as e:
            last = e
    raise last

def l2norm(X: np.ndarray) -> np.ndarray:
    return X / (np.linalg.norm(X, axis=1, keepdims=True) + 1e-12)

def _coerce_content_id_column(df: pd.DataFrame) -> pd.Series:
    """여러 이름/형태로 저장된 content_id를 최대한 복구."""
    df = df.copy()
    orig_cols = list(df.columns)
    df.columns = [c.strip() for c in df.columns]
    name_map = {c.lower(): c for c in df.columns}
    for key in ["content_id", "contentid", "content id", "id", "unnamed: 0"]:
        if key in name_map:
            cand = name_map[key]
            s = df[cand]
            if s.dtype == object:
                s = s.astype(str).str.strip()
            s = pd.to_numeric(s, errors="coerce").astype("Int64")
            if s.isna().all():
                continue
            return s
    raise RuntimeError(f"content_id 열을 찾지 못했습니다. 원본 컬럼: {orig_cols}")

def load_item_embeddings() -> Tuple[pd.DataFrame, np.ndarray, List[str]]:
    """아이템 임베딩 로드 (content_id + emb_*), numpy 행렬, emb 컬럼 이름 리턴"""
    i1 = read_csv_retry(ITEM_EMB_1)
    if i1 is not None and not i1.empty:
        df = i1; src = os.path.basename(ITEM_EMB_1)
    else:
        i2 = read_csv_retry(ITEM_EMB_2)
        if i2 is None or i2.empty:
            raise RuntimeError("item_embeddings CSV 필요: item_embeddings_torch.csv 또는 item_embeddings.csv")
        df = i2; src = os.path.basename(ITEM_EMB_2)

    df = df.copy()
    df.columns = [c.strip() for c in df.columns]
    df["content_id"] = _coerce_content_id_column(df)

    emb_cols = [c for c in df.columns if c.lower().startswith("emb_")]
    if not emb_cols:
        alt = [c for c in df.columns if re.search(r"\d+$", c)]
        raise RuntimeError(f"임베딩 컬럼(emb_*) 없음. 확인 필요. 후보: {alt}")

    for c in emb_cols:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    df = df.dropna(subset=emb_cols + ["content_id"]).copy()
    df["content_id"] = df["content_id"].astype(int)

    if EXCLUDE_DUP_CONTENTS:
        df = df.drop_duplicates(subset=["content_id"], keep="first").reset_index(drop=True)

    if not ASSUME_NORMALIZED:
        df.loc[:, emb_cols] = l2norm(df[emb_cols].to_numpy(np.float32))

    M = df[emb_cols].to_numpy(np.float32)
    M = l2norm(M)  # 안전하게 한 번 더 정규화
    df.loc[:, emb_cols] = M

    print(f"✅ 아이템 임베딩 로드: {src} (items={len(df)}, dim={len(emb_cols)})")
    return df[["content_id"] + emb_cols], M, emb_cols

def load_content_names() -> Optional[pd.DataFrame]:
    """
    contents.csv 에서 content_id, content_name, domain 을 뽑아서 리턴.
    content_name 후보 컬럼:
      content_name > master_title > original_title > title > name
    domain 컬럼은 'domain'(대소문자 무시) 사용.
    """
    if not os.path.exists(CONTENTS):
        print("ℹ️ contents.csv 없음 → 이름/도메인 없이 진행")
        return None

    meta = read_csv_retry(CONTENTS)
    if meta is None or meta.empty:
        print("ℹ️ contents.csv 비어있음 → 이름/도메인 없이 진행")
        return None

    meta = meta.copy()
    meta.columns = [c.strip() for c in meta.columns]

    if "content_id" not in meta.columns:
        print("ℹ️ contents.csv 에 content_id 없음 → 이름/도메인 붙이기 생략")
        return None
    meta["content_id"] = pd.to_numeric(meta["content_id"], errors="coerce").astype("Int64")
    meta = meta[meta["content_id"].notna()].copy()
    meta["content_id"] = meta["content_id"].astype(int)

    lower_map = {c.lower(): c for c in meta.columns}

    # 이름 컬럼
    name_key = None
    for key in ["content_name", "master_title", "original_title", "title", "name"]:
        if key in lower_map:
            name_key = lower_map[key]
            break

    # 도메인 컬럼
    domain_key = lower_map.get("domain")

    if name_key is None and domain_key is None:
        print("ℹ️ contents.csv 에서 이름/도메인 컬럼을 찾지 못함 → 메타 없이 진행")
        return None

    use_cols = ["content_id"]
    rename_map = {}
    if name_key is not None:
        use_cols.append(name_key)
        rename_map[name_key] = "content_name"
    if domain_key is not None:
        use_cols.append(domain_key)
        rename_map[domain_key] = "domain"

    meta = meta[use_cols].drop_duplicates(subset=["content_id"], keep="first")
    meta = meta.rename(columns=rename_map)

    used_cols_str = ", ".join([c for c in ["content_name", "domain"] if c in meta.columns])
    print(f"✅ contents 메타 사용: {used_cols_str}")
    return meta

def load_content_genres_raw() -> pd.DataFrame:
    """
    content_raw_genres.csv 에서
      content_id, raw_genre, g_norm(lower)
    를 리턴.
    (없으면 추천 자체를 중단)
    """
    if not os.path.exists(RAW_GENRES):
        raise RuntimeError("content_raw_genres.csv 가 없습니다. 장르 기반 추천 불가.")

    gdf = read_csv_retry(RAW_GENRES)
    if gdf is None or gdf.empty:
        raise RuntimeError("content_raw_genres.csv 가 비어있습니다. 장르 기반 추천 불가.")

    gdf = gdf.copy()
    gdf.columns = [c.strip() for c in gdf.columns]

    if not {"content_id", "raw_genre"}.issubset(set(gdf.columns)):
        raise RuntimeError("content_raw_genres.csv 에 content_id/raw_genre 컬럼이 필요합니다.")

    gdf["content_id"] = pd.to_numeric(gdf["content_id"], errors="coerce").astype("Int64")
    gdf = gdf[gdf["content_id"].notna()].copy()
    gdf["content_id"] = gdf["content_id"].astype(int)

    gdf["raw_genre"] = gdf["raw_genre"].astype(str).str.strip()
    gdf = gdf[gdf["raw_genre"] != ""].copy()

    gdf["g_norm"] = gdf["raw_genre"].str.lower().str.strip()

    if gdf.empty:
        raise RuntimeError("content_raw_genres.csv 에 유효한 장르가 없습니다. 장르 기반 추천 불가.")

    print(f"✅ 콘텐츠 장르 raw 로드: rows={len(gdf)}, unique_genres={gdf['g_norm'].nunique()}")
    return gdf[["content_id", "raw_genre", "g_norm"]]

def build_genre_centroids(
    item_df: pd.DataFrame,
    M: np.ndarray,
    gdf_raw: pd.DataFrame
) -> Dict[str, np.ndarray]:
    """
    각 장르(g_norm)별로 해당 장르를 가진 아이템 임베딩 평균 → 장르 센트로이드.
    (item_df / M 은 이미 '장르가 있는 아이템'만 남겨진 상태라고 가정)
    """
    idx_map = {cid: i for i, cid in enumerate(item_df["content_id"].tolist())}
    genre_vecs: Dict[str, np.ndarray] = {}
    for g, grp in gdf_raw.groupby("g_norm"):
        idxs = [idx_map[c] for c in grp["content_id"].tolist() if c in idx_map]
        if not idxs:
            continue
        v = M[idxs].mean(axis=0)
        v = v / (np.linalg.norm(v) + 1e-12)
        genre_vecs[g] = v
    print(f"✅ 장르 센트로이드 수: {len(genre_vecs)}")
    return genre_vecs

def build_genres_agg(gdf_raw: pd.DataFrame) -> pd.DataFrame:
    """
    content_id 별 genres_str ('장르1|장르2|...') 생성.
    """
    agg = (
        gdf_raw.groupby("content_id")["raw_genre"]
        .apply(lambda x: "|".join(pd.unique(x.tolist())))
        .reset_index()
        .rename(columns={"raw_genre": "genres_str"})
    )
    print(f"✅ 장르 집계 매핑: {len(agg)}개 content_id")
    return agg

def load_user_genre_prefs() -> pd.DataFrame:
    """
    user_preferred_genres.csv (+ optional user_preferences.csv)를 읽어서
      user_id, username, genres(list of str, g_norm) 를 리턴.
    """
    upg = read_csv_retry(UP_GENRES)
    if upg is None or upg.empty:
        raise RuntimeError("user_preferred_genres.csv 필요.")

    upg = upg.copy()
    upg.columns = [c.strip() for c in upg.columns]

    # 장르 컬럼 찾기
    lower_map = {c.lower(): c for c in upg.columns}
    genre_col = None
    for key in ["genre", "raw_genre"]:
        if key in lower_map:
            genre_col = lower_map[key]
            break
    if genre_col is None:
        raise RuntimeError("user_preferred_genres.csv 에 genre/raw_genre 컬럼이 필요합니다.")

    upg["genre_norm"] = upg[genre_col].astype(str).str.strip().str.lower()
    upg = upg[upg["genre_norm"] != ""].copy()

    # 1) user_preferences.csv 기반 (preference_id -> id, username)
    upref = read_csv_retry(UPREF)
    rows = []

    if upref is not None and not upref.empty and \
       "id" in [c.lower() for c in upref.columns] and \
       "preference_id" in [c.lower() for c in upg.columns]:

        upref = upref.copy()
        upref.columns = [c.strip() for c in upref.columns]
        low_map_pref = {c.lower(): c for c in upref.columns}
        id_col = low_map_pref.get("id")
        username_col = next((low_map_pref[c] for c in low_map_pref if c in ["username", "user_name"]), None)

        low_map_upg = {c.lower(): c for c in upg.columns}
        pref_id_col = low_map_upg["preference_id"]

        upg[pref_id_col] = pd.to_numeric(upg[pref_id_col], errors="coerce").astype("Int64")
        upg = upg[upg[pref_id_col].notna()].copy()

        for _, ur in upref.iterrows():
            pid = ur[id_col]
            try:
                pid_int = int(pid)
            except Exception:
                continue
            uname = str(ur[username_col]) if username_col else ""
            g_list = upg.loc[upg[pref_id_col] == pid_int, "genre_norm"].tolist()
            if not g_list:
                continue
            g_list = sorted(set(g_list))
            rows.append([pid_int, uname, g_list])
    else:
        # 2) user_preferred_genres.csv 안에서 user_id / username 찾기
        low_map = {c.lower(): c for c in upg.columns}
        user_key = None
        for key in ["user_id", "userid", "user id", "id"]:
            if key in low_map:
                user_key = low_map[key]
                break
        username_key = None
        for key in ["username", "user_name"]:
            if key in low_map:
                username_key = low_map[key]
                break

        if user_key is not None:
            upg[user_key] = pd.to_numeric(upg[user_key], errors="coerce").astype("Int64")
            upg = upg[upg[user_key].notna()].copy()
            upg[user_key] = upg[user_key].astype(int)

            for uid, grp in upg.groupby(user_key):
                uname = ""
                if username_key:
                    uname = str(grp[username_key].iloc[0])
                g_list = sorted(set(grp["genre_norm"].tolist()))
                rows.append([int(uid), uname, g_list])
        elif username_key is not None:
            # username만 있을 때: 내부 user_id를 1,2,3...으로 할당
            for new_id, (uname, grp) in enumerate(upg.groupby(username_key), start=1):
                g_list = sorted(set(grp["genre_norm"].tolist()))
                rows.append([new_id, str(uname), g_list])
        else:
            raise RuntimeError("user_preferred_genres.csv 에 user_id나 username 정보가 없습니다.")

    if not rows:
        raise RuntimeError("유저 선호 장르 정보가 비어 있습니다.")

    prefs = pd.DataFrame(rows, columns=["user_id", "username", "genres"])
    print(f"✅ 유저 선호 장르 로드: users={len(prefs)}")
    return prefs

def main():
    # 1) 아이템 임베딩
    item_df, M, emb_cols = load_item_embeddings()

    # 2) 콘텐츠 장르 raw (필수) + '장르가 있는 아이템만' 필터링
    gdf_raw = load_content_genres_raw()  # 여기서 없으면 바로 에러
    has_genre_ids = set(gdf_raw["content_id"].unique())

    # ▶ 장르 없는 콘텐츠는 여기서 제거
    before_cnt = len(item_df)
    mask = item_df["content_id"].isin(has_genre_ids)
    item_df = item_df[mask].reset_index(drop=True)
    M = M[mask.to_numpy()]  # 임베딩 행렬도 같이 필터링
    after_cnt = len(item_df)
    print(f"✅ 장르가 있는 아이템만 유지: {before_cnt} → {after_cnt}")

    if after_cnt == 0:
        raise RuntimeError("장르가 있는 아이템이 하나도 없습니다. 추천 불가.")

    item_ids = item_df["content_id"].to_numpy(int)

    # 3) 장르 집계 + 장르 센트로이드
    genres_agg = build_genres_agg(gdf_raw)
    genre_vecs = build_genre_centroids(item_df, M, gdf_raw)

    # 4) 유저별 선호 장르
    prefs = load_user_genre_prefs()

    # 5) content_name/domain 붙일 메타
    meta_name = load_content_names()

    # 6) 글로벌 평균 벡터 (fallback) - 이미 장르 있는 아이템만 남은 상태
    v_global = M.mean(axis=0)
    v_global = v_global / (np.linalg.norm(v_global) + 1e-12)

    rec_rows = []
    for _, ur in prefs.iterrows():
        uid = int(ur["user_id"])
        uname = str(ur.get("username", "") or "")
        g_list: List[str] = ur["genres"] or []

        # 이 유저의 선호 장르 중 센트로이드가 존재하는 것만 사용
        g_match = [g for g in g_list if g in genre_vecs]
        if g_match:
            g_vecs = np.stack([genre_vecs[g] for g in g_match], axis=0)
            v_user = g_vecs.mean(axis=0)
            v_user = v_user / (np.linalg.norm(v_user) + 1e-12)
        else:
            # 해당 장르 센트로이드가 하나도 없으면 글로벌 평균 (그래도 후보는 장르 있는 아이템만)
            v_user = v_global

        # 코사인 유사도 (M, v_user 모두 정규화된 상태)
        sims = M @ v_user.reshape(-1, 1)
        sims = sims.reshape(-1)

        k = min(TOPK, len(sims))
        if k == 0:
            continue
        part = np.argpartition(-sims, k-1)[:k]
        order = part[np.argsort(-sims[part])]
        top_ids = item_ids[order]
        top_scs = sims[order]

        for rank, (cid, sc) in enumerate(zip(top_ids, top_scs), start=1):
            rec_rows.append([uid, uname, rank, int(cid), float(sc)])

    rec = pd.DataFrame(rec_rows, columns=["user_id", "username", "rank", "content_id", "score"])

    # 7) 이름 + 도메인 붙이기
    if meta_name is not None:
        rec = rec.merge(meta_name, on="content_id", how="left")

    # 8) 장르 붙이기 (집계)
    if genres_agg is not None:
        rec = rec.merge(genres_agg, on="content_id", how="left")

    # 9) 컬럼 순서 정리
    cols = ["user_id", "username", "rank", "content_id"]
    if "content_name" in rec.columns:
        cols.append("content_name")
    if "domain" in rec.columns:
        cols.append("domain")
    if "genres_str" in rec.columns:
        cols.append("genres_str")
    cols.append("score")
    rec = rec[cols]

    rec.sort_values(["user_id", "rank"]).to_csv(OUT_RECS, index=False, encoding="utf-8-sig")
    print(f"✅ recommendations_topK_genre.csv 저장: {OUT_RECS} (rows={len(rec)})")

if __name__ == "__main__":
    main()


✅ 아이템 임베딩 로드: item_embeddings_torch.csv (items=495, dim=64)


RuntimeError: content_raw_genres.csv 에 content_id/raw_genre 컬럼이 필요합니다.

# 장르기반 추천인데 최신성 필터 추가

In [68]:
# -*- coding: utf-8 -*-

# 장르 기반 유저-아이템 Top-K 추천 (user_embeddings.csv 사용 X)
# ▶ 장르가 없는 콘텐츠는 추천 후보에서 제외
# ▶ contents.csv의 created_at 기준으로 2020년 이후 콘텐츠만 사용
#
# 입력:
#   clean/item_embeddings_torch.csv 또는 item_embeddings.csv (content_id, emb_0..)
#   clean/contents.csv                        (선택) content_id + 제목/도메인/created_at 컬럼
#   clean/content_raw_genres.csv              (필수) content_id, raw_genre
#   clean/user_preferred_genres.csv           (유저별 선호 장르)
#   clean/user_preferences.csv                (선택) id, username 등 (있으면 username 매핑)
#
# 출력:
#   clean/recommendations_topK_genre.csv
#   (user_id, username, rank, content_id, content_name, domain, genres_str, score)

import os
import re
import numpy as np
import pandas as pd
from typing import Optional, List, Dict, Tuple, Set

BASE        = r"csv 데이터\clean"
ITEM_EMB_1  = os.path.join(BASE, "item_embeddings_torch.csv")
ITEM_EMB_2  = os.path.join(BASE, "item_embeddings.csv")
CONTENTS    = os.path.join(BASE, "contents.csv")             # 이름/도메인/created_at 메타
RAW_GENRES  = os.path.join(BASE, "content_raw_genres.csv")   # 장르 메타
UPREF       = os.path.join(BASE, "user_preferences.csv")     # (선택) user_id, username
UP_GENRES   = os.path.join(BASE, "user_preferred_genres.csv")# 유저 선호 장르
OUT_RECS    = os.path.join(BASE, "recommendations_topK_genre_2020.csv")

TOPK = 5
EXCLUDE_DUP_CONTENTS = True   # content_id 중복행이 있으면 첫 번째만 사용
ASSUME_NORMALIZED    = True   # 임베딩이 이미 L2 정규화되어 있다면 True, 아니면 False로
MIN_YEAR_CREATED     = 2020   # created_at 기준 필터 (2020년 이후만 사용)


def read_csv_retry(path, encodings=("utf-8-sig","utf-8","cp949","euc-kr","latin1"), **kw) -> Optional[pd.DataFrame]:
    if not os.path.exists(path):
        return None
    last = None
    for enc in encodings:
        try:
            return pd.read_csv(path, encoding=enc, **kw)
        except Exception as e:
            last = e
    raise last


def l2norm(X: np.ndarray) -> np.ndarray:
    return X / (np.linalg.norm(X, axis=1, keepdims=True) + 1e-12)


def _coerce_content_id_column(df: pd.DataFrame) -> pd.Series:
    """여러 이름/형태로 저장된 content_id를 최대한 복구."""
    df = df.copy()
    orig_cols = list(df.columns)
    df.columns = [c.strip() for c in df.columns]
    name_map = {c.lower(): c for c in df.columns}
    for key in ["content_id", "contentid", "content id", "id", "unnamed: 0"]:
        if key in name_map:
            cand = name_map[key]
            s = df[cand]
            if s.dtype == object:
                s = s.astype(str).str.strip()
            s = pd.to_numeric(s, errors="coerce").astype("Int64")
            if s.isna().all():
                continue
            return s
    raise RuntimeError(f"content_id 열을 찾지 못했습니다. 원본 컬럼: {orig_cols}")


def load_allowed_ids_by_created_at(min_year: int = MIN_YEAR_CREATED) -> Optional[Set[int]]:
    """
    contents.csv 의 created_at 컬럼을 이용해서
    지정 연도 이상(min_year)인 content_id 집합을 리턴.
    - contents.csv / created_at / content_id 가 없으면 None 리턴 (필터 미적용)
    - 조건을 만족하는 콘텐츠가 하나도 없으면 RuntimeError 발생
    """
    if not os.path.exists(CONTENTS):
        print("ℹ️ contents.csv 없음 → created_at 필터 생략")
        return None

    df = read_csv_retry(CONTENTS)
    if df is None or df.empty:
        print("ℹ️ contents.csv 비어있음 → created_at 필터 생략")
        return None

    df = df.copy()
    df.columns = [c.strip() for c in df.columns]

    if "created_at" not in df.columns:
        print("ℹ️ contents.csv 에 created_at 컬럼 없음 → created_at 필터 생략")
        return None

    # content_id 추출
    try:
        df["content_id"] = _coerce_content_id_column(df)
    except Exception as e:
        print("ℹ️ contents.csv 에서 content_id 추출 실패 → created_at 필터 생략:", e)
        return None

    df = df[df["content_id"].notna()].copy()
    df["content_id"] = df["content_id"].astype(int)

    # created_at 파싱
    df["created_at"] = pd.to_datetime(df["created_at"], errors="coerce")
    df = df[df["created_at"].notna()].copy()

    # min_year 이상만 남기기
    df = df[df["created_at"].dt.year >= min_year]
    allowed_ids = set(df["content_id"].tolist())

    print(f"✅ created_at >= {min_year} 인 콘텐츠 수: {len(allowed_ids)}")
    if not allowed_ids:
        raise RuntimeError(f"created_at >= {min_year} 조건을 만족하는 콘텐츠가 없습니다.")
    return allowed_ids


def load_item_embeddings() -> Tuple[pd.DataFrame, np.ndarray, List[str]]:
    """아이템 임베딩 로드 (content_id + emb_*), numpy 행렬, emb 컬럼 이름 리턴"""
    i1 = read_csv_retry(ITEM_EMB_1)
    if i1 is not None and not i1.empty:
        df = i1; src = os.path.basename(ITEM_EMB_1)
    else:
        i2 = read_csv_retry(ITEM_EMB_2)
        if i2 is None or i2.empty:
            raise RuntimeError("item_embeddings CSV 필요: item_embeddings_torch.csv 또는 item_embeddings.csv")
        df = i2; src = os.path.basename(ITEM_EMB_2)

    df = df.copy()
    df.columns = [c.strip() for c in df.columns]
    df["content_id"] = _coerce_content_id_column(df)

    emb_cols = [c for c in df.columns if c.lower().startswith("emb_")]
    if not emb_cols:
        alt = [c for c in df.columns if re.search(r"\d+$", c)]
        raise RuntimeError(f"임베딩 컬럼(emb_*) 없음. 확인 필요. 후보: {alt}")

    for c in emb_cols:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    df = df.dropna(subset=emb_cols + ["content_id"]).copy()
    df["content_id"] = df["content_id"].astype(int)

    if EXCLUDE_DUP_CONTENTS:
        df = df.drop_duplicates(subset=["content_id"], keep="first").reset_index(drop=True)

    if not ASSUME_NORMALIZED:
        df.loc[:, emb_cols] = l2norm(df[emb_cols].to_numpy(np.float32))

    M = df[emb_cols].to_numpy(np.float32)
    M = l2norm(M)  # 안전하게 한 번 더 정규화
    df.loc[:, emb_cols] = M

    print(f"✅ 아이템 임베딩 로드: {src} (items={len(df)}, dim={len(emb_cols)})")
    return df[["content_id"] + emb_cols], M, emb_cols


def load_content_names() -> Optional[pd.DataFrame]:
    """
    contents.csv 에서 content_id, content_name, domain 을 뽑아서 리턴.
    content_name 후보 컬럼:
      content_name > master_title > original_title > title > name
    domain 컬럼은 'domain'(대소문자 무시) 사용.
    """
    if not os.path.exists(CONTENTS):
        print("ℹ️ contents.csv 없음 → 이름/도메인 없이 진행")
        return None

    meta = read_csv_retry(CONTENTS)
    if meta is None or meta.empty:
        print("ℹ️ contents.csv 비어있음 → 이름/도메인 없이 진행")
        return None

    meta = meta.copy()
    meta.columns = [c.strip() for c in meta.columns]

    if "content_id" not in meta.columns:
        print("ℹ️ contents.csv 에 content_id 없음 → 이름/도메인 붙이기 생략")
        return None
    meta["content_id"] = pd.to_numeric(meta["content_id"], errors="coerce").astype("Int64")
    meta = meta[meta["content_id"].notna()].copy()
    meta["content_id"] = meta["content_id"].astype(int)

    lower_map = {c.lower(): c for c in meta.columns}

    # 이름 컬럼
    name_key = None
    for key in ["content_name", "master_title", "original_title", "title", "name"]:
        if key in lower_map:
            name_key = lower_map[key]
            break

    # 도메인 컬럼
    domain_key = lower_map.get("domain")

    if name_key is None and domain_key is None:
        print("ℹ️ contents.csv 에서 이름/도메인 컬럼을 찾지 못함 → 메타 없이 진행")
        return None

    use_cols = ["content_id"]
    rename_map = {}
    if name_key is not None:
        use_cols.append(name_key)
        rename_map[name_key] = "content_name"
    if domain_key is not None:
        use_cols.append(domain_key)
        rename_map[domain_key] = "domain"

    meta = meta[use_cols].drop_duplicates(subset=["content_id"], keep="first")
    meta = meta.rename(columns=rename_map)

    used_cols_str = ", ".join([c for c in ["content_name", "domain"] if c in meta.columns])
    print(f"✅ contents 메타 사용: {used_cols_str}")
    return meta


def load_content_genres_raw() -> pd.DataFrame:
    """
    content_raw_genres.csv 에서
      content_id, raw_genre, g_norm(lower)
    를 리턴.
    (없으면 추천 자체를 중단)
    """
    if not os.path.exists(RAW_GENRES):
        raise RuntimeError("content_raw_genres.csv 가 없습니다. 장르 기반 추천 불가.")

    gdf = read_csv_retry(RAW_GENRES)
    if gdf is None or gdf.empty:
        raise RuntimeError("content_raw_genres.csv 가 비어있습니다. 장르 기반 추천 불가.")

    gdf = gdf.copy()
    gdf.columns = [c.strip() for c in gdf.columns]

    if not {"content_id", "raw_genre"}.issubset(set(gdf.columns)):
        raise RuntimeError("content_raw_genres.csv 에 content_id/raw_genre 컬럼이 필요합니다.")

    gdf["content_id"] = pd.to_numeric(gdf["content_id"], errors="coerce").astype("Int64")
    gdf = gdf[gdf["content_id"].notna()].copy()
    gdf["content_id"] = gdf["content_id"].astype(int)

    gdf["raw_genre"] = gdf["raw_genre"].astype(str).str.strip()
    gdf = gdf[gdf["raw_genre"] != ""].copy()

    gdf["g_norm"] = gdf["raw_genre"].str.lower().str.strip()

    if gdf.empty:
        raise RuntimeError("content_raw_genres.csv 에 유효한 장르가 없습니다. 장르 기반 추천 불가.")

    print(f"✅ 콘텐츠 장르 raw 로드: rows={len(gdf)}, unique_genres={gdf['g_norm'].nunique()}")
    return gdf[["content_id", "raw_genre", "g_norm"]]


def build_genre_centroids(
    item_df: pd.DataFrame,
    M: np.ndarray,
    gdf_raw: pd.DataFrame
) -> Dict[str, np.ndarray]:
    """
    각 장르(g_norm)별로 해당 장르를 가진 아이템 임베딩 평균 → 장르 센트로이드.
    (item_df / M 은 이미 '장르가 있는 아이템'만 남겨진 상태라고 가정)
    """
    idx_map = {cid: i for i, cid in enumerate(item_df["content_id"].tolist())}
    genre_vecs: Dict[str, np.ndarray] = {}
    for g, grp in gdf_raw.groupby("g_norm"):
        idxs = [idx_map[c] for c in grp["content_id"].tolist() if c in idx_map]
        if not idxs:
            continue
        v = M[idxs].mean(axis=0)
        v = v / (np.linalg.norm(v) + 1e-12)
        genre_vecs[g] = v
    print(f"✅ 장르 센트로이드 수: {len(genre_vecs)}")
    return genre_vecs


def build_genres_agg(gdf_raw: pd.DataFrame) -> pd.DataFrame:
    """
    content_id 별 genres_str ('장르1|장르2|...') 생성.
    """
    agg = (
        gdf_raw.groupby("content_id")["raw_genre"]
        .apply(lambda x: "|".join(pd.unique(x.tolist())))
        .reset_index()
        .rename(columns={"raw_genre": "genres_str"})
    )
    print(f"✅ 장르 집계 매핑: {len(agg)}개 content_id")
    return agg


def load_user_genre_prefs() -> pd.DataFrame:
    """
    user_preferred_genres.csv (+ optional user_preferences.csv)를 읽어서
      user_id, username, genres(list of str, g_norm) 를 리턴.
    """
    upg = read_csv_retry(UP_GENRES)
    if upg is None or upg.empty:
        raise RuntimeError("user_preferred_genres.csv 필요.")

    upg = upg.copy()
    upg.columns = [c.strip() for c in upg.columns]

    # 장르 컬럼 찾기
    lower_map = {c.lower(): c for c in upg.columns}
    genre_col = None
    for key in ["genre", "raw_genre"]:
        if key in lower_map:
            genre_col = lower_map[key]
            break
    if genre_col is None:
        raise RuntimeError("user_preferred_genres.csv 에 genre/raw_genre 컬럼이 필요합니다.")

    upg["genre_norm"] = upg[genre_col].astype(str).str.strip().str.lower()
    upg = upg[upg["genre_norm"] != ""].copy()

    # 1) user_preferences.csv 기반 (preference_id -> id, username)
    upref = read_csv_retry(UPREF)
    rows = []

    if upref is not None and not upref.empty and \
       "id" in [c.lower() for c in upref.columns] and \
       "preference_id" in [c.lower() for c in upg.columns]:

        upref = upref.copy()
        upref.columns = [c.strip() for c in upref.columns]
        low_map_pref = {c.lower(): c for c in upref.columns}
        id_col = low_map_pref.get("id")
        username_col = next((low_map_pref[c] for c in low_map_pref if c in ["username", "user_name"]), None)

        low_map_upg = {c.lower(): c for c in upg.columns}
        pref_id_col = low_map_upg["preference_id"]

        upg[pref_id_col] = pd.to_numeric(upg[pref_id_col], errors="coerce").astype("Int64")
        upg = upg[upg[pref_id_col].notna()].copy()

        for _, ur in upref.iterrows():
            pid = ur[id_col]
            try:
                pid_int = int(pid)
            except Exception:
                continue
            uname = str(ur[username_col]) if username_col else ""
            g_list = upg.loc[upg[pref_id_col] == pid_int, "genre_norm"].tolist()
            if not g_list:
                continue
            g_list = sorted(set(g_list))
            rows.append([pid_int, uname, g_list])
    else:
        # 2) user_preferred_genres.csv 안에서 user_id / username 찾기
        low_map = {c.lower(): c for c in upg.columns}
        user_key = None
        for key in ["user_id", "userid", "user id", "id"]:
            if key in low_map:
                user_key = low_map[key]
                break
        username_key = None
        for key in ["username", "user_name"]:
            if key in low_map:
                username_key = low_map[key]
                break

        if user_key is not None:
            upg[user_key] = pd.to_numeric(upg[user_key], errors="coerce").astype("Int64")
            upg = upg[upg[user_key].notna()].copy()
            upg[user_key] = upg[user_key].astype(int)

            for uid, grp in upg.groupby(user_key):
                uname = ""
                if username_key:
                    uname = str(grp[username_key].iloc[0])
                g_list = sorted(set(grp["genre_norm"].tolist()))
                rows.append([int(uid), uname, g_list])
        elif username_key is not None:
            # username만 있을 때: 내부 user_id를 1,2,3...으로 할당
            for new_id, (uname, grp) in enumerate(upg.groupby(username_key), start=1):
                g_list = sorted(set(grp["genre_norm"].tolist()))
                rows.append([new_id, str(uname), g_list])
        else:
            raise RuntimeError("user_preferred_genres.csv 에 user_id나 username 정보가 없습니다.")

    if not rows:
        raise RuntimeError("유저 선호 장르 정보가 비어 있습니다.")

    prefs = pd.DataFrame(rows, columns=["user_id", "username", "genres"])
    print(f"✅ 유저 선호 장르 로드: users={len(prefs)}")
    return prefs


def main():
    # 1) 아이템 임베딩
    item_df, M, emb_cols = load_item_embeddings()

    # 2) 콘텐츠 장르 raw (필수)
    gdf_raw = load_content_genres_raw()  # 여기서 없으면 바로 에러

    # 2-1) created_at 기준으로 2020년 이후 콘텐츠만 허용 (옵션)
    allowed_ids = load_allowed_ids_by_created_at()  # 없으면 None
    if allowed_ids is not None:
        before_g = len(gdf_raw)
        gdf_raw = gdf_raw[gdf_raw["content_id"].isin(allowed_ids)].reset_index(drop=True)
        print(f"✅ created_at >= {MIN_YEAR_CREATED} 필터 적용 (장르 테이블): {before_g} → {len(gdf_raw)}")
        if gdf_raw.empty:
            raise RuntimeError(f"created_at >= {MIN_YEAR_CREATED} 이면서 장르가 있는 콘텐츠가 없습니다. 추천 불가.")

    # 2-2) '장르가 있는 아이템만' 아이템 테이블에서 유지
    has_genre_ids = set(gdf_raw["content_id"].unique())

    before_cnt = len(item_df)
    mask = item_df["content_id"].isin(has_genre_ids)
    item_df = item_df[mask].reset_index(drop=True)
    M = M[mask.to_numpy()]  # 임베딩 행렬도 같이 필터링
    after_cnt = len(item_df)
    print(f"✅ 장르+날짜 조건 만족 아이템만 유지: {before_cnt} → {after_cnt}")

    if after_cnt == 0:
        raise RuntimeError("조건을 만족하는 아이템이 하나도 없습니다. 추천 불가.")

    item_ids = item_df["content_id"].to_numpy(int)

    # 3) 장르 집계 + 장르 센트로이드
    genres_agg = build_genres_agg(gdf_raw)
    genre_vecs = build_genre_centroids(item_df, M, gdf_raw)

    # 4) 유저별 선호 장르
    prefs = load_user_genre_prefs()

    # 5) content_name/domain 붙일 메타
    meta_name = load_content_names()

    # 6) 글로벌 평균 벡터 (fallback) - 이미 장르+날짜 조건 만족 아이템만 남은 상태
    v_global = M.mean(axis=0)
    v_global = v_global / (np.linalg.norm(v_global) + 1e-12)

    rec_rows = []
    for _, ur in prefs.iterrows():
        uid = int(ur["user_id"])
        uname = str(ur.get("username", "") or "")
        g_list: List[str] = ur["genres"] or []

        # 이 유저의 선호 장르 중 센트로이드가 존재하는 것만 사용
        g_match = [g for g in g_list if g in genre_vecs]
        if g_match:
            g_vecs = np.stack([genre_vecs[g] for g in g_match], axis=0)
            v_user = g_vecs.mean(axis=0)
            v_user = v_user / (np.linalg.norm(v_user) + 1e-12)
        else:
            # 해당 장르 센트로이드가 하나도 없으면 글로벌 평균 (그래도 후보는 장르 있는 아이템만)
            v_user = v_global

        # 코사인 유사도 (M, v_user 모두 정규화된 상태)
        sims = M @ v_user.reshape(-1, 1)
        sims = sims.reshape(-1)

        k = min(TOPK, len(sims))
        if k == 0:
            continue
        part = np.argpartition(-sims, k-1)[:k]
        order = part[np.argsort(-sims[part])]
        top_ids = item_ids[order]
        top_scs = sims[order]

        for rank, (cid, sc) in enumerate(zip(top_ids, top_scs), start=1):
            rec_rows.append([uid, uname, rank, int(cid), float(sc)])

    rec = pd.DataFrame(rec_rows, columns=["user_id", "username", "rank", "content_id", "score"])

    # 7) 이름 + 도메인 붙이기
    if meta_name is not None:
        rec = rec.merge(meta_name, on="content_id", how="left")

    # 8) 장르 붙이기 (집계)
    if genres_agg is not None:
        rec = rec.merge(genres_agg, on="content_id", how="left")

    # 9) 컬럼 순서 정리
    cols = ["user_id", "username", "rank", "content_id"]
    if "content_name" in rec.columns:
        cols.append("content_name")
    if "domain" in rec.columns:
        cols.append("domain")
    if "genres_str" in rec.columns:
        cols.append("genres_str")
    cols.append("score")
    rec = rec[cols]

    rec.sort_values(["user_id", "rank"]).to_csv(OUT_RECS, index=False, encoding="utf-8-sig")
    print(f"✅ recommendations_topK_genre.csv 저장: {OUT_RECS} (rows={len(rec)})")


if __name__ == "__main__":
    main()


✅ 아이템 임베딩 로드: item_embeddings_torch.csv (items=495, dim=64)


RuntimeError: content_raw_genres.csv 에 content_id/raw_genre 컬럼이 필요합니다.

# 도메인 별 추천 아이템 생성

In [39]:
# -*- coding: utf-8 -*-

#도메인별 Top-N 추천 (유저/아이템 임베딩 + contents.csv)
#입력:
#  clean/user_embeddings.csv                  (user_id, username, emb_0..)
#  clean/item_embeddings_torch.csv 또는 item_embeddings.csv (content_id, emb_0..)
#  clean/contents.csv                         (content_id, domain, title/name/...)
#출력:
#  clean/recommendations_per_domain.csv       (user_id, username, rank, content_id, content_title, score, domain)

import os, re
import numpy as np
import pandas as pd
from typing import Optional, Dict

# ===== 경로 =====
BASE = r"csv 데이터\clean"
USER_EMB   = os.path.join(BASE, "user_embeddings.csv")
ITEM_EMB_1 = os.path.join(BASE, "item_embeddings_torch.csv")
ITEM_EMB_2 = os.path.join(BASE, "item_embeddings.csv")
CONTENTS   = os.path.join(BASE, "contents.csv")
OUT_CSV    = os.path.join(BASE, "recommendations_per_domain.csv")

# ===== 설정 =====
DOMAIN_QUOTA: Dict[str, int] = {"AV": 3, "GAME": 3, "WEBNOVEL": 3}   # 도메인별 개수
FILL_WITH_GLOBAL = True    # 도메인 후보 부족 시 글로벌 Top으로 보충
ASSUME_NORMALIZED = True   # 임베딩이 이미 L2 정규화라면 True
EXCLUDE_DUP_CONTENTS = True

# ===== 유틸 =====
def read_csv_retry(path, encodings=("utf-8-sig","utf-8","cp949","euc-kr","latin1"), **kw) -> Optional[pd.DataFrame]:
    if not os.path.exists(path):
        return None
    last=None
    for enc in encodings:
        try:
            return pd.read_csv(path, encoding=enc, **kw)
        except Exception as e:
            last=e
    raise last

def l2norm(X: np.ndarray) -> np.ndarray:
    return X / (np.linalg.norm(X, axis=1, keepdims=True) + 1e-12)

def _coerce_content_id_column(df: pd.DataFrame) -> pd.Series:
    df = df.copy()
    orig_cols = list(df.columns)
    df.columns = [c.strip() for c in df.columns]
    name_map = {c.lower(): c for c in df.columns}
    for key in ["content_id", "contentid", "content id", "id", "unnamed: 0"]:
        if key in name_map:
            s = df[name_map[key]]
            if s.dtype == object:
                s = s.astype(str).str.strip()
            s = pd.to_numeric(s, errors="coerce").astype("Int64")
            if not s.isna().all():
                return s
    raise RuntimeError(f"content_id 열을 찾지 못했습니다. 원본 컬럼: {orig_cols}")

def load_user_embeddings() -> pd.DataFrame:
    u = read_csv_retry(USER_EMB)
    if u is None or u.empty:
        raise RuntimeError("user_embeddings.csv 필요.")
    u = u.copy()
    u.columns = [c.strip() for c in u.columns]
    if "user_id" not in u.columns:
        if "id" in u.columns:
            u = u.rename(columns={"id": "user_id"})
        else:
            raise RuntimeError("user_embeddings.csv에 user_id 컬럼 필요.")
    u["user_id"] = pd.to_numeric(u["user_id"], errors="coerce").astype("Int64")
    u = u[u["user_id"].notna()].copy()
    u["user_id"] = u["user_id"].astype(int)

    emb_cols = [c for c in u.columns if c.lower().startswith("emb_")]
    if not emb_cols:
        raise RuntimeError("user_embeddings.csv에 emb_로 시작하는 임베딩 컬럼이 없습니다.")
    for c in emb_cols:
        u[c] = pd.to_numeric(u[c], errors="coerce")
    u.dropna(subset=emb_cols, inplace=True)
    if not ASSUME_NORMALIZED:
        u.loc[:, emb_cols] = l2norm(u[emb_cols].to_numpy(np.float32))

    # username 컬럼 유연 처리
    uname_col = None
    for c in u.columns:
        if c.lower() == "username":
            uname_col = c; break
    if uname_col is None:
        u["username"] = ""
        uname_col = "username"

    return u[["user_id", uname_col] + emb_cols].rename(columns={uname_col: "username"})

def load_item_embeddings() -> pd.DataFrame:
    i1 = read_csv_retry(ITEM_EMB_1)
    if i1 is not None and not i1.empty:
        df = i1; src = os.path.basename(ITEM_EMB_1)
    else:
        i2 = read_csv_retry(ITEM_EMB_2)
        if i2 is None or i2.empty:
            raise RuntimeError("item_embeddings CSV 필요: item_embeddings_torch.csv 또는 item_embeddings.csv")
        df = i2; src = os.path.basename(ITEM_EMB_2)

    df = df.copy()
    df.columns = [c.strip() for c in df.columns]
    df["content_id"] = _coerce_content_id_column(df)

    emb_cols = [c for c in df.columns if c.lower().startswith("emb_")]
    if not emb_cols:
        alt = [c for c in df.columns if re.search(r"\d+$", c)]
        raise RuntimeError(f"임베딩 컬럼(emb_*) 없음. 확인 필요. 후보: {alt}")

    for c in emb_cols:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    df = df.dropna(subset=emb_cols + ["content_id"]).copy()
    df["content_id"] = df["content_id"].astype(int)

    if EXCLUDE_DUP_CONTENTS:
        df = df.drop_duplicates(subset=["content_id"], keep="first").reset_index(drop=True)

    if not ASSUME_NORMALIZED:
        df.loc[:, emb_cols] = l2norm(df[emb_cols].to_numpy(np.float32))

    print(f"✅ 아이템 임베딩 로드: {src} (items={len(df)}, dim={len(emb_cols)})")
    return df[["content_id"] + emb_cols]

# 제목 컬럼 자동 탐지: title / name / content_name / ko_title / ko_name 등
def detect_title_column(df: pd.DataFrame) -> Optional[str]:
    cand_list = ["title", "name", "content_name", "ko_title", "ko_name", "master_title"]
    cols_lower = {c.lower(): c for c in df.columns}
    for c in cand_list:
        if c in cols_lower:
            return cols_lower[c]
    return None


def main():
    # --- 데이터 로드 ---
    u = load_user_embeddings()
    i = load_item_embeddings()

    contents = read_csv_retry(CONTENTS)
    if contents is None or contents.empty or {"content_id","domain"} - set(contents.columns):
        raise RuntimeError("contents.csv 필요 (content_id, domain).")
    contents = contents.copy()
    contents.columns = [c.strip() for c in contents.columns]

    # content_id 정리
    contents["content_id"] = pd.to_numeric(contents["content_id"], errors="coerce").astype("Int64")
    contents = contents[contents["content_id"].notna()].copy()
    contents["content_id"] = contents["content_id"].astype(int)

    # ★ 제목 컬럼 찾기
    title_col = detect_title_column(contents)
    if title_col is None:
        raise RuntimeError("contents.csv에서 제목 컬럼(title/name/...)을 찾지 못했습니다. (타이틀 없는 콘텐츠 제외를 위해 제목 컬럼이 필요합니다.)")
    else:
        print(f"✅ 제목 컬럼 발견: {title_col}")

    # ★ 제목이 비어있는 콘텐츠 제외
    contents[title_col] = contents[title_col].astype(str).str.strip()
    contents = contents[contents[title_col] != ""].copy()

    # ★ 제목 있는 콘텐츠의 content_id만 유효
    valid_ids = set(contents["content_id"].unique())

    # ★ 아이템 임베딩도 valid_ids만 남기기 → 타이틀 없는 콘텐츠는 추천 후보에서 제거
    i = i[i["content_id"].isin(valid_ids)].reset_index(drop=True)

    # 도메인 맵은 필터링된 contents 기준
    dom_map = contents.set_index("content_id")["domain"].to_dict()

    ucols = [c for c in u.columns if c.lower().startswith("emb_")]
    icols = [c for c in i.columns if c.lower().startswith("emb_")]

    U = u[ucols].to_numpy(np.float32); U = l2norm(U)   # 안전하게 한 번 더 정규화
    I = i[icols].to_numpy(np.float32); I = l2norm(I)
    item_ids = i["content_id"].to_numpy(int)

    rec_rows = []
    for r in range(U.shape[0]):
        uvec = U[r:r+1, :]                   # [1, D]
        sims = (I @ uvec.T).reshape(-1)      # [N,]

        # 1) 도메인별 quota 채우기
        picked_mask = np.zeros_like(sims, dtype=bool)
        per_domain_picks = []
        for dom, need in DOMAIN_QUOTA.items():
            if need <= 0: 
                continue
            # 해당 도메인 아이템 마스크
            dom_mask = np.array([dom_map.get(int(cid)) == dom for cid in item_ids], dtype=bool)
            if not dom_mask.any():
                continue
            # 아직 안 뽑힌 + 해당 도메인
            cand_mask = dom_mask & (~picked_mask)
            if not cand_mask.any():
                continue
            cand_idx = np.where(cand_mask)[0]
            cand_scores = sims[cand_idx]

            k = min(need, len(cand_idx))
            part = np.argpartition(-cand_scores, k-1)[:k]
            order = part[np.argsort(-cand_scores[part])]
            chosen = cand_idx[order]

            picked_mask[chosen] = True
            for j in chosen:
                per_domain_picks.append((int(item_ids[j]), float(sims[j]), dom))

        # 2) 부족분 글로벌 Top으로 보충(옵션)
        total_need = sum(max(0, n) for n in DOMAIN_QUOTA.values())
        if FILL_WITH_GLOBAL and picked_mask.sum() < total_need:
            remain = total_need - picked_mask.sum()
            left_idx = np.where(~picked_mask)[0]
            if len(left_idx) > 0:
                left_scores = sims[left_idx]
                k2 = min(remain, len(left_idx))
                part = np.argpartition(-left_scores, k2-1)[:k2]
                order = part[np.argsort(-left_scores[part])]

                chosen = left_idx[order]
                for j in chosen:
                    c = int(item_ids[j])
                    per_domain_picks.append((c, float(sims[j]), dom_map.get(c, "")))
                    picked_mask[j] = True

        # 3) rank 매기고 저장
        per_domain_picks.sort(key=lambda x: -x[1])
        uid = int(u.loc[r, "user_id"]); uname = str(u.loc[r, "username"])
        for rank, (cid, sc, dom) in enumerate(per_domain_picks, start=1):
            rec_rows.append([uid, uname, rank, cid, sc, dom])

    rec = pd.DataFrame(rec_rows, columns=["user_id","username","rank","content_id","score","domain"])

    # 제목 join 해서 content_title 컬럼 추가 (여기서 contents는 이미 "제목 있는 것만"이라, 빈 타이틀 없음)
    name_df = contents[["content_id", title_col]].drop_duplicates("content_id")
    rec = rec.merge(name_df, on="content_id", how="left")
    rec = rec.rename(columns={title_col: "content_title"})

    # 저장 컬럼 순서
    rec = rec.sort_values(["user_id","rank"])
    rec_cols = ["user_id","username","rank","content_id","content_title","score","domain"]
    rec[rec_cols].to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
    print(f"✅ recommendations_per_domain.csv 저장: {OUT_CSV} (rows={len(rec)})")

    # 콘솔 요약
    try:
        for uid in sorted(rec["user_id"].unique()):
            sub = rec[rec["user_id"]==uid].sort_values("rank")
            print(f"\n[User {uid}] 총 {len(sub)}개")
            for _, row in sub.iterrows():
                title = ""
                if isinstance(row["content_title"], str) and row["content_title"]:
                    title = f"  name={row['content_title']}"
                print(f"  #{int(row['rank']):2d}  cid={int(row['content_id'])}  dom={row['domain']}  sim={row['score']:.4f}{title}")
    except Exception:
        pass

if __name__ == "__main__":
    main()


✅ 아이템 임베딩 로드: item_embeddings_torch.csv (items=495, dim=64)
✅ 제목 컬럼 발견: master_title
✅ recommendations_per_domain.csv 저장: csv 데이터\clean\recommendations_per_domain.csv (rows=90)

[User 1] 총 9개
  # 1  cid=29  dom=AV  sim=0.4297  name=적인걸 3: 사대천왕
  # 2  cid=49  dom=AV  sim=0.3864  name=에브리씽 에브리웨어 올 앳 원스
  # 3  cid=21  dom=AV  sim=0.3615  name=베놈: 라스트 댄스
  # 4  cid=485  dom=WEBNOVEL  sim=0.2978  name=영역 침범
  # 5  cid=503  dom=WEBNOVEL  sim=0.2955  name=광고천재의 연예계 공략법
  # 6  cid=235  dom=GAME  sim=0.2890  name=Disciples II: Rise of the Elves
  # 7  cid=547  dom=WEBNOVEL  sim=0.2634  name=플레이어 2세로 살아가는 법
  # 8  cid=295  dom=GAME  sim=0.2545  name=Typer Shark! Deluxe
  # 9  cid=245  dom=GAME  sim=0.2274  name=Wolfenstein 3D

[User 2] 총 9개
  # 1  cid=76  dom=AV  sim=0.4220  name=옥스퍼드에서의 날들
  # 2  cid=207  dom=GAME  sim=0.4073  name=Half-Life
  # 3  cid=193  dom=AV  sim=0.3437  name=보잭 홀스맨
  # 4  cid=519  dom=WEBNOVEL  sim=0.3338  name=처맞으며 레벨업
  # 5  cid=474  dom=WEBNOVEL  sim=0.3119  name=파혼 후